# 확률 네트워크기반 친환경 자동차 충전소(GVCS) 로케이션 인텔리전스(Location Intelligence) 대시보드

## 제안 배경

**친환경 자동차 생산에만 관심이 쏠려 사용자를 고려하지 못한 설치로 인한 잉여 충전소 발생**

- 민관 협력을 통해 충전 서비스 관련 사업이 꾸준히 증가함. 하지만 전국에 설치된 공공 급속충전기 총 2,896개(2021년 기준) 중 하루 1회 이하로 사용한 충전기가 1,164개로 전체의 40.1%를 차지하고 있다. 심지어 17개는 전혀 사용하지 않은 것으로 나타남.

- 이와 같은 충전소마다 이용 편차가 큰 이유는 전기차 보급 목표에 따라 구역별로  충전소를 나눠 설치했기 때문이라는 지적도 있다.

- 한국전기차충전협동조합은 앞으로 수요가 몰리는 거점을 중심을 중심으로 충전기 설치를 해야하 한다는 의견 주장함.

- 기존의 정부 중심으로 공급자 입장에서 설치하던 충전소를 친환경 자동차 차주를 고려해 이용자 중심으로 변화 설치가 필요하다는 지적도 있다.

즉, **꾸준히 증가하는 친환경 차량의 수요를 위해 저비용 고효율의 충전소 설치를 목표 한다.** 충전소 설치 사업자의 기대 수익 최적화를 위해 사용자 입장을 고려한 체계적이며 최적의 입지 선정 결과를 제시함.

## 사용 데이터

1. 충전소 위치 및 현황 정보
2. 급속충전기 보급 현황
3. LPG충전소 현황
4. 오존 월별 대기오염도
5. 아황산가스 월별 대기오염도
6. 이산화질소 월별 대기오염도
7. 미세먼지 월별 대기오염도
8. 행정구역별 인구수
9. 항만 현황
10. 지역별 전기차 등록대 수

In [191]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import osmnx as ox
import networkx as nx
import numpy as np
from scipy.stats import norm
import plotly.express as px
import plotly.graph_objects as go
import pgmpy as pg
from pgmpy.factors.discrete import JointProbabilityDistribution as JPD

### 데이터 적합성 판단 기준

In [192]:
# 환경 요인 판단 기준
o3_standard = 0.003     # 오존
so2_standard = 0.002    # 아황산가스
no2_standard = 0.012    # 이산화질소    
co_standard = 0.3       # 일산화탄소
pm10_standard = 15      # 미세먼지
pm25_standard = 10      # 초미세먼지
khai_standard = 50      # 통합대기환경지수


# 사회 요인 판단 기준
busan_people_standard = 200000  # 고정인구
foottraffic_standard = 400000   # 유동인구
elec_vehicle_standard = 50000   # 전기차 수
hidro_vehicle_standard = 1500   # 수소차 수
LPG_standard = 2                # LPG 충전소 수
EVCS_standard = 500             # 전기차 충전소 수
HVCS_standard = 15              # 수소차 충전소 수
intersection_standard = 100     # 교차로 수
EV_distance_standard = 0.5      # 주변충전소(전기) 최단거리(km)
HV_distance_standard = 5        # 주변충전소(수소) 최단거리(km)
highway_standard_E = 1000       # 주변고속도로 최단거리(전기)(m)
highway_standard_H = 3          # 주변고속도로 최단거리(수소)(km)

# 경제 요인 판단 기준
EVCS_cost_standard = 14_000_000
HVCS_construction_cost_standard = 3000000000    # 수소충전소 구축 비용
LPG_land_standard = 6000000000                  # LPG 충전소 토지비용
parking_area_standard = 100                     # 주차구획 수

# 기술 요인 판단 기준
charger_pro_default = 0.38           # 충전 방식

In [193]:
def advanced_replace(df, col, str, regex):
    '''
    DataFrame 특정 열의 특수문자 제거
    :param df: 대상 DataFrame
    :param col: column
    :param str: 제거할 특수 문자.
    :param regex: 적용할 정규표현식
    :return:
    '''
    df[col] = df[col].replace(str,0)
    df[col] = df[col].replace(to_replace=regex, value=r'', regex=True)
    return df

In [194]:
def show_norm(mean,std,min,max):
     '''
     정규분포 함수를 matplotlib을 통해 시각화
     mean: 평균값
     std: 표준편차
     min: 관측된 데이터의 최솟값
     max: 관측된 데이터의 최댓값
     '''
     min = min - std
     # x값 설정(min에서 max까지 unit만큼씩 증가하는 x)
     x = np.linspace(min,max,100)
     # y값은 평균이 mean이고 표준편차가 std인 확률밀도함수
     fig = px.line(x=x,y=norm.pdf(x,loc=mean,scale=std))
     fig.show()

In [195]:
def cal_norm(mean,std,min,max,value,affect):
     '''
     정규분포에서 value에 대한 누적확률 구하기
     mean: 평균값
     std: 표준편차
     min: 관측된 데이터의 최솟값
     max: 관측된 데이터의 최댓값
     value: 누적확률을 구하고자 하는 데이터의 측정값
     affect: 입지에 긍정/부정적인 영향(True/False)
     '''
     min = min - std
     red ='rgb(239,85,59)'
     blue = 'rgb(100,110,250)'
     fig = go.Figure()
     
     # 확률 값을 구할 특정 구간의 범위 설정
     cum_a = np.linspace(min,value,100)
     cum_b = np.linspace(value,max,100)
     
     pro = norm(mean,std).cdf(value)-norm(mean,std).cdf(min).round(3)
     if affect == True:
          # 구간 사이에 색을 채움
          fig.add_trace(go.Scatter(x=cum_a,y=norm.pdf(cum_a,mean,std), fill = 'tozeroy',name='적합',line=dict(color=blue)))
          fig.add_trace(go.Scatter(x=cum_b,y=norm.pdf(cum_b,mean,std), fill = 'tozeroy',name='부적합',line=dict(color=red)))
          fig.update_yaxes(visible=False)
          annotations = []
          annotations.append(dict(x=value, y=norm.pdf(value,loc=mean,scale=std), showarrow=False, text=round(pro,3), font=dict(size=15,color=blue), xshift=-40, yshift=-100, bordercolor=blue,borderwidth=2))
          annotations.append(dict(x=value, y=norm.pdf(value,loc=mean,scale=std), showarrow=False, text=round(1-pro,3),font=dict(size=15,color=red),xshift=40, yshift=-100, bordercolor=red,borderwidth=2))
     else:
          fig.add_trace(go.Scatter(x=cum_a,y=norm.pdf(cum_a,mean,std), fill = 'tozeroy',name='부적합',line=dict(color=red)))
          fig.add_trace(go.Scatter(x=cum_b,y=norm.pdf(cum_b,mean,std), fill = 'tozeroy',name='적합',line=dict(color=blue)))
          fig.update_yaxes(visible=False)
          annotations = []
          annotations.append(dict(x=value, y=norm.pdf(value,loc=mean,scale=std), showarrow=False, text=round(pro,3), font=dict(size=15,color=red), xshift=-40, yshift=-100, bordercolor=red,borderwidth=2))
          annotations.append(dict(x=value, y=norm.pdf(value,loc=mean,scale=std), showarrow=False, text=round(1-pro,3),font=dict(size=15,color=blue),xshift=40, yshift=-100, bordercolor=blue,borderwidth=2))     
          pro = 1 - pro
     fig.update_layout(annotations=annotations)
     # value까지의 누적확률에서 min까지의 누적확률을 뺌
     # fig.show()

     # 최종 누적확률 반환
     return pro, 1-pro

In [196]:
# 데이터의 수를 통한 확률 계산
def cal_pro(dataframe,column,standard,affect):
    if dataframe[column].dtypes == 'dtype(\'O\')':
        num = dataframe[dataframe[column]==standard].shape[0]
        total_num = dataframe.shape[0]
    else:
        if affect == True:
            num = dataframe[dataframe[column]>=standard].shape[0]
            total_num = dataframe.shape[0]
        else:
            num = dataframe[dataframe[column]<standard].shape[0]
            total_num = dataframe.shape[0]

    probability = num/total_num

    return probability, 1-probability

## 상위 요소 - 환경적 요인

대기오염도를 기반하여 확률 값을 계산한다.

### 대기오염도 데이터 불러오기

In [197]:
air_pollution_file_path = "C:\\Users\\SAMSUNG\\Desktop\\Green-Car-Charging-Station-Site-Selection\\Data\\대기오염시도별실시간측정정보(환경부).csv"
df_air_pollution = pd.read_csv(air_pollution_file_path, encoding='cp949')
df_air_pollution

,sidoName,stationName,mangName,so2Value,coValue,o3Value,no2Value,pm10Value,pm25Value,khaiValue
0,충북,송정동(봉명동),도시대기,0.002,0.6,0.003,0.022,65.0,12.0,56.0
1,충북,사천동,도시대기,0.002,0.5,0.002,0.019,28.0,22.0,59.0
2,충북,용담동,도시대기,NaN,0.4,0.002,0.018,12.0,11.0,30.0
3,충북,용암동,도시대기,0.002,0.5,0.002,0.017,31.0,15.0,52.0
4,충북,복대동,도로변대기,0.004,0.7,0.002,0.028,25.0,10.0,47.0
...,...,...,...,...,...,...,...,...,...,...
10907,강원,묵호항,항만,0.013,1.3,0.003,0.034,33.0,16.0,56.0
10908,강원,철원(DMZ),교외대기,0.001,0.3,0.023,0.010,28.0,27.0,82.0
10909,강원,화천(DMZ),교외대기,0.001,0.3,0.033,0.007,16.0,15.0,64.0
10910,강원,인제(DMZ),교외대기,0.001,0.2,0.038,0.005,10.0,8.0,57.0


### 결측값 보간

In [198]:
df_air_pollution = df_air_pollution.interpolate(method='pad')
df_air_pollution

,sidoName,stationName,mangName,so2Value,coValue,o3Value,no2Value,pm10Value,pm25Value,khaiValue
0,충북,송정동(봉명동),도시대기,0.002,0.6,0.003,0.022,65.0,12.0,56.0
1,충북,사천동,도시대기,0.002,0.5,0.002,0.019,28.0,22.0,59.0
2,충북,용담동,도시대기,0.002,0.4,0.002,0.018,12.0,11.0,30.0
3,충북,용암동,도시대기,0.002,0.5,0.002,0.017,31.0,15.0,52.0
4,충북,복대동,도로변대기,0.004,0.7,0.002,0.028,25.0,10.0,47.0
...,...,...,...,...,...,...,...,...,...,...
10907,강원,묵호항,항만,0.013,1.3,0.003,0.034,33.0,16.0,56.0
10908,강원,철원(DMZ),교외대기,0.001,0.3,0.023,0.010,28.0,27.0,82.0
10909,강원,화천(DMZ),교외대기,0.001,0.3,0.033,0.007,16.0,15.0,64.0
10910,강원,인제(DMZ),교외대기,0.001,0.2,0.038,0.005,10.0,8.0,57.0


### 부산 대기오염 데이터 추출

In [199]:
df_air_pollution = df_air_pollution[df_air_pollution['sidoName']=='부산']
df_air_pollution.reset_index(drop=True,inplace=True)
df_air_pollution

,sidoName,stationName,mangName,so2Value,coValue,o3Value,no2Value,pm10Value,pm25Value,khaiValue
0,부산,광복동,도시대기,0.002,0.3,0.016,0.018,11.0,8.0,30.0
1,부산,초량동,도로변대기,0.002,0.4,0.010,0.028,16.0,5.0,47.0
2,부산,태종대,도시대기,0.003,0.2,0.019,0.020,15.0,10.0,33.0
3,부산,청학동,도시대기,0.003,0.2,0.021,0.018,10.0,7.0,35.0
4,부산,전포동,도시대기,0.002,0.2,0.016,0.014,14.0,10.0,37.0
...,...,...,...,...,...,...,...,...,...,...
625,부산,하단동,도시대기,0.003,0.4,0.015,0.029,43.0,23.0,66.0
626,부산,하단동,도시대기,0.003,0.3,0.023,0.015,31.0,22.0,59.0
627,부산,하단동,도시대기,0.002,0.5,0.005,0.031,25.0,17.0,51.0
628,부산,하단동,도시대기,0.004,0.5,0.002,0.047,27.0,17.0,78.0


#### 부산시 오존 적합도

In [200]:
o3_pro,o3_pro_negative = cal_norm(df_air_pollution['o3Value'].mean(),df_air_pollution['o3Value'].std(),
df_air_pollution['o3Value'].min(),df_air_pollution['o3Value'].max(),o3_standard,False)
o3_pro

0.8017202428389887

#### 부산시 아황산가스 적합도

In [201]:
so2_pro,so2_pro_negative = cal_norm(df_air_pollution['so2Value'].mean(),df_air_pollution['so2Value'].std(),
df_air_pollution['so2Value'].min(),df_air_pollution['so2Value'].max(),so2_standard,False)
so2_pro

0.7911203783441816

#### 부산시 이산화질소 적합도

In [202]:
no2_pro,no2_pro_negative = cal_norm(df_air_pollution['no2Value'].mean(),df_air_pollution['no2Value'].std(),
df_air_pollution['no2Value'].min(),df_air_pollution['no2Value'].max(),no2_standard,False)
no2_pro

0.909590433996196

#### 부산시 일산화탄소 적합도

In [203]:
co_pro,co_pro_negative = cal_norm(df_air_pollution['coValue'].mean(),df_air_pollution['coValue'].std(),
df_air_pollution['coValue'].min(),df_air_pollution['coValue'].max(),co_standard,False)
co_pro

0.8287531282163694

#### 부산시 미세먼지 적합도

In [204]:
pm10_pro,pm10_pro_negative = cal_norm(df_air_pollution['pm10Value'].mean(),df_air_pollution['pm10Value'].std(),
df_air_pollution['pm10Value'].min(),df_air_pollution['pm10Value'].max(),pm10_standard,False)
pm10_pro

0.8866811666335976

In [205]:
pm25_pro,pm25_pro_negative = cal_norm(df_air_pollution['pm25Value'].mean(),df_air_pollution['pm25Value'].std(),
df_air_pollution['pm25Value'].min(),df_air_pollution['pm25Value'].max(),pm25_standard,False)
pm25_pro

0.8415688322677302

#### 부산시 통합대기환경 지수 적합도

In [206]:
khai_pro,khai_pro_negative = cal_norm(df_air_pollution['khaiValue'].mean(),df_air_pollution['khaiValue'].std(),
df_air_pollution['khaiValue'].min(),df_air_pollution['khaiValue'].max(),khai_standard,False)
khai_pro

0.8220286272881268

## 상위 요소 - 사회적 요인

### 행정구역 인구 데이터 불러오기

In [207]:
people_file_path = 'C:\\Users\\SAMSUNG\\Desktop\\Green-Car-Charging-Station-Site-Selection\\Data\\행정구역 인구 데이터.csv'
df_people = pd.read_csv(people_file_path, encoding='cp949', header=1)
df_people

,행정구역(시군구)별,총인구수 (명)
0,전국,51592660
1,서울특별시,9500480
2,종로구,144005
3,중구,122087
4,용산구,222009
...,...,...
287,거창군,60756
288,합천군,42613
289,제주특별자치도,677413
290,제주시,493076


### 부산광역시 인구 데이터 추출

In [208]:
df_busan_people = df_people[28:44].sort_values('행정구역(시군구)별')
df_busan_people.set_index('행정구역(시군구)별',inplace=True)
df_busan_people['총인구수 (명)'] = df_busan_people['총인구수 (명)'].astype(float)
df_busan_people

,총인구수 (명)
행정구역(시군구)별,
강서구,143638.0
금정구,225033.0
기장군,176505.0
남구,260617.0
동구,88257.0
동래구,273976.0
부산진구,351022.0
북구,280852.0
사상구,206365.0


In [209]:
df_busan_people.describe()

,총인구수 (명)
count,16.000000
mean,208763.375000
std,98697.592227
min,40585.000000
25%,135080.250000
50%,206521.000000
75%,275695.000000
max,393130.000000


#### 부산시 고정인구 적합도

In [210]:
busan_people_pro,busan_people_pro_negative = cal_norm(df_busan_people['총인구수 (명)'].mean(), df_busan_people['총인구수 (명)'].std(), df_busan_people['총인구수 (명)'].min(), 
df_busan_people['총인구수 (명)'].max(), busan_people_standard, False)

### 유동인구 데이터 불러오기

In [211]:
foottraffic_file_path = 'C:\\Users\\SAMSUNG\\Desktop\\Green-Car-Charging-Station-Site-Selection\\Data\\부산 유동인구 데이터.csv'
df_foottraffic = pd.read_csv(foottraffic_file_path, encoding='cp949')
df_foottraffic

,구군,1월,2월,3월,4월,5월,6월,7월,8월,9월,10월,11월,12월,월 평균
0,강서구,"325,284.39","323,049.87","335,120.06","340,570.52","332,289.77","340,678.45","337,488.64","328,374.74","328,764.38","333,414.26","340,405.10","337,753.15","333,599.44"
1,금정구,"482,598.89","477,047.11","484,669.59","490,864.46","487,435.45","484,936.00","483,898.58","482,964.23","487,082.06","484,633.90","485,502.59","488,392.99","485,002.15"
2,기장군,"341,181.17","343,398.79","342,489.52","345,184.20","348,793.68","348,500.32","350,763.40","349,328.59","348,077.97","348,449.77","345,905.89","344,245.29","346,359.88"
3,남구,"539,072.58","526,693.91","524,611.67","524,961.53","526,758.54","531,078.89","529,810.42","530,199.16","531,658.35","525,870.37","528,106.40","531,610.18","529,202.67"
4,동구,"185,466.33","185,459.30","189,082.61","192,702.25","189,819.14","191,618.73","189,864.16","189,368.74","190,328.67","191,182.03","194,732.77","193,341.86","190,247.22"
5,동래구,"553,655.76","549,718.60","543,091.96","535,439.53","533,908.61","528,124.07","527,730.64","525,468.79","532,656.70","527,498.20","520,126.56","524,788.88","533,517.36"
6,부산진구,"757,261.10","760,742.04","760,212.63","761,728.33","765,460.09","764,636.01","759,638.81","754,837.78","763,942.81","761,731.75","763,818.34","768,957.14","761,913.90"
7,북구,"559,133.62","554,946.17","543,084.57","543,479.25","543,145.91","540,381.96","542,289.98","546,905.11","551,394.74","542,887.88","539,732.97","545,952.52","546,111.22"
8,사상구,"496,102.34","492,871.78","493,054.71","496,932.63","494,094.59","494,207.66","492,973.61","490,296.85","493,842.47","491,808.21","493,815.86","495,297.44","493,774.85"
9,사하구,"641,541.35","632,045.91","628,461.73","633,805.98","630,333.83","629,635.14","630,260.69","631,377.03","633,402.47","627,551.97","625,380.25","631,475.21","631,272.63"


In [212]:
foottraffic_col = df_foottraffic.iloc[:,1:-1].columns.tolist()

In [213]:
df_foottraffic.drop(columns=foottraffic_col, inplace=True)
df_foottraffic.set_index('구군',inplace=True)
df_foottraffic

,월 평균
구군,
강서구,"333,599.44"
금정구,"485,002.15"
기장군,"346,359.88"
남구,"529,202.67"
동구,"190,247.22"
동래구,"533,517.36"
부산진구,"761,913.90"
북구,"546,111.22"
사상구,"493,774.85"


In [214]:
advanced_replace(df_foottraffic, '월 평균','-',r'[^0-9.0-9]')
df_foottraffic['월 평균'] = df_foottraffic['월 평균'].astype(float)

In [215]:
df_foottraffic.describe()

,월 평균
count,16.000000
mean,441344.991250
std,205145.538509
min,121290.850000
25%,311221.577500
50%,440973.920000
75%,536665.825000
max,867841.630000


In [216]:
foottraffic_pro,foottraffic_pro_negative = cal_norm(df_foottraffic['월 평균'].mean(),df_foottraffic['월 평균'].std(),
df_foottraffic['월 평균'].min(),df_foottraffic['월 평균'].max(),foottraffic_standard,False)
foottraffic_pro


0.5848617448890694

### 친환경 자동차 등록수 데이터 불러오기

In [217]:
vehicle_file_path = 'C:\\Users\\SAMSUNG\\Desktop\\Green-Car-Charging-Station-Site-Selection\\Data\\연료별 자동차 등록 수.csv'
df_vehicle = pd.read_csv(vehicle_file_path, encoding='cp949')
drop_col = df_vehicle.iloc[:,1:3].columns.tolist()
df_vehicle.drop(columns=drop_col,inplace=True)
df_vehicle

,연료,서울,부산,대구,인천,광주,대전,울산,세종,경기,강원,충북,충남,전북,전남,경북,경남,제주,계
0,휘발유,"1,649,518","714,221","584,780","818,874","312,081","341,412","297,231","94,929","3,056,798","365,724","403,958","525,588","398,182","466,016","654,776","886,361","302,434","11,872,883"
1,경유,"1,077,183","550,411","449,206","627,029","266,237","247,724","217,553","66,126","2,450,214","371,277","380,402","516,001","430,676","547,999","664,042","736,135","253,205","9,851,420"
2,엘피지,"240,940","111,371","106,654","113,654","88,883","67,456","36,101","12,985","404,449","58,335","70,304","89,210","93,615","111,494","127,818","139,592","58,412","1,931,273"
3,전기,"45,133","16,059","18,076","15,191","6,233","9,550","3,948","2,180","50,111","9,117","10,421","12,142","8,931","10,658","13,018","14,957","26,931","272,656"
4,하이브리드(휘발유+전기),"147,022","61,167","48,320","90,275","24,091","24,167","19,593","11,034","256,643","24,001","25,842","32,669","25,448","38,154","36,999","56,219","30,057","951,701"
5,하이브리드(경유+전기),"2,656","4,187","2,146","7,672",464,393,194,141,"3,964",286,580,461,694,486,405,"1,774",341,"26,844"
6,하이브리드(LPG+전기),"1,601",735,806,696,628,481,457,153,"3,453",611,639,728,785,833,974,"1,034",335,"14,949"
7,기타연료,"7,172","16,097","4,545","13,915","4,036","3,830","5,205","1,208","29,809","4,376","6,918","9,574","5,182","9,666","10,974","9,161",886,"142,554"
8,수소,"2,435","1,473",352,"1,139","1,018","1,045","2,395",205,"4,004","1,959","1,095",908,"1,435",448,77,"1,899",5,"21,892"
9,총합,"3,173,660","1,475,721","1,214,885","1,688,445","703,671","696,058","582,677","188,961","6,259,445","835,686","900,159","1,187,281","964,948","1,185,754","1,509,083","1,847,132","672,606","25,086,172"


In [218]:
# 전국 자동차 수 통계값 구하기 (전기, 수소)
col = df_vehicle.iloc[:,1:].columns.tolist()
for c in col:
    advanced_replace(df_vehicle,c,'-',r'[^0-9.0-9]')
    df_vehicle[c] = df_vehicle[c].astype(int)
df_elec_vehicle = df_vehicle.iloc[3:7,:-1].sum()
df_elec_vehicle = df_elec_vehicle[1:]

df_elec_vehicle = df_elec_vehicle.astype(int)
df_elec_vehicle.describe()


count        17.000000
mean      74479.411765
std       75151.087588
min       13508.000000
25%       34591.000000
50%       50131.000000
75%       73984.000000
max      314171.000000
dtype: float64

In [219]:
df_hidro_vehicle = df_vehicle.iloc[8,1:-1]
df_hidro_vehicle = df_hidro_vehicle.astype(int)
df_hidro_vehicle.describe()

count      17.000000
mean     1287.764706
std      1030.779531
min         5.000000
25%       448.000000
50%      1095.000000
75%      1899.000000
max      4004.000000
Name: 8, dtype: float64

#### 부산시 전기차, 수소차 수 적합도

In [220]:
elec_vehicle_pro,elec_vehicle_pro_negative = cal_norm(df_elec_vehicle.mean(), df_elec_vehicle.std(),
 df_elec_vehicle.min(), df_elec_vehicle.max(),elec_vehicle_standard,False)
print(elec_vehicle_pro)

hidro_vehicle_pro,hidro_vehicle_pro_negative = cal_norm(df_hidro_vehicle.mean(), df_hidro_vehicle.std(),
 df_hidro_vehicle.min(), df_hidro_vehicle.max(),hidro_vehicle_standard,False)
print(hidro_vehicle_pro)

0.6626879332080854
0.4304353506049742


### LPG 충전소 현황 데이터 불러오기 (수소차)

In [290]:
LPG_file_path = 'C:\\Users\\SAMSUNG\\Desktop\\Green-Car-Charging-Station-Site-Selection\\Data\\부산 LPG 충전소 현황(한국가스안전공사).csv'
df_LPG = pd.read_csv(LPG_file_path, encoding='cp949')
df_LPG

,행정시,행정구,업소명,소재지,관리구분,위도,경도,면적,공시지가,토지비용,주변충전소 최단거리,주변고속도로 최단거리,교차로 수,환경적요인,사회적요인,경제적요인,기술적요인,최종확률
0,부산,강서구,개인택시신공항충전소,부산 강서구 경전철로188번길 95,자동차충전,35.198669,128.966594,2545.0,2144000.0,5.456480e+09,6.175720,0.815596,52,0.838573,0.501317,0.768969,1,0.707003
1,부산,강서구,부산공항LPG충전소,부산 강서구 공항로 459,자동차충전,35.149064,128.951891,3105.0,1174000.0,3.645270e+09,3.270797,5.631472,16,0.838573,0.301317,0.768969,1,0.627003
2,부산,강서구,신항만에너지,부산 강서구 낙동남로 507,자동차충전,35.112947,128.875260,3105.0,1174000.0,3.645270e+09,2.907968,11.968669,76,0.838573,0.301317,0.768969,1,0.627003
3,부산,강서구,(주)지원네트웍스 부산강서LPG충전소,부산 강서구 낙동북로 310,자동차충전,35.212507,128.962572,1396.0,1140000.0,1.591440e+09,7.752939,0.864172,32,0.838573,0.501317,0.768969,1,0.707003
4,부산,강서구,신명지에너지,부산 강서구 르노삼성대로 560,자동차충전,35.097554,128.927351,481.0,700300.0,3.368443e+08,7.676891,11.748654,159,0.838573,0.551317,0.768969,1,0.727003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,부산,연제구,부산개인택시엘피지충전소,부산 연제구 월드컵대로 399,자동차충전,35.186013,129.055974,9212.0,688600.0,6.343383e+09,7.648635,5.612178,146,0.838573,0.651317,0.268969,1,0.617003
59,부산,영도구,(주)태종에너지,부산 영도구 영선대로 41,자동차충전,35.084996,129.040033,1610.2,2679000.0,4.313726e+09,8.277014,13.118878,255,0.838573,0.551317,0.768969,1,0.727003
60,부산,영도구,부산에너지,부산 영도구 해양로 160,용기+자동차+탱크+13kg용기,35.091275,129.069484,5801.0,989400.0,5.739509e+09,9.604391,13.821871,67,0.838573,0.401317,0.768969,1,0.667003
61,부산,영도구,개인택시 영도LPG충전소,부산 영도구 해양로 296,자동차충전,35.082176,129.076429,1818.0,349800.0,6.359364e+08,10.740567,14.353595,98,0.838573,0.401317,0.768969,1,0.667003


In [222]:
# 복합충전소를 위해 1500 m^2이상의 부지가 요구됨
df_LPG = df_LPG[df_LPG['면적'] >= 1500].reset_index(drop=True)
df_LPG

,행정시,행정구,업소명,소재지,관리구분,위도,경도,면적,공시지가,토지비용,주변충전소 최단거리,주변고속도로 최단거리,교차로 수
0,부산,강서구,개인택시신공항충전소,부산 강서구 경전철로188번길 95,자동차충전,35.198669,128.966594,2545.0,2144000.0,5.456480e+09,6.175720,0.815596,52
1,부산,강서구,부산공항LPG충전소,부산 강서구 공항로 459,자동차충전,35.149064,128.951891,3105.0,1174000.0,3.645270e+09,3.270797,5.631472,16
2,부산,강서구,신항만에너지,부산 강서구 낙동남로 507,자동차충전,35.112947,128.875260,3105.0,1174000.0,3.645270e+09,2.907968,11.968669,76
3,부산,강서구,르노삼성자동차(주),부산 강서구 르노삼성대로 61,자동차충전,35.095408,128.883215,1538.0,640100.0,9.844738e+08,4.134222,13.975751,82
4,부산,강서구,죽림LPG충전소,부산 강서구 호계로 102,자동차충전,35.200736,128.893462,1580.0,1017000.0,1.606860e+09,10.512855,2.696444,16
5,부산,강서구,가락충전소,부산 강서구 호계로 195,자동차충전,35.208847,128.890148,2600.0,1017000.0,2.644200e+09,11.295108,1.747535,20
6,부산,금정구,(주)해연,부산 금정구 개좌로 225-15,용기+탱크+13kg용기,35.230796,129.122388,3346.0,1370000.0,4.584020e+09,12.992938,3.145687,119
7,부산,금정구,(주)신진가스,부산 금정구 반송로 433,자동차충전,35.215006,129.111143,1646.4,2797000.0,4.604981e+09,13.435925,3.870264,407
8,부산,기장군,마라톤가스(주)청강충전소,부산 기장군 기장읍 기장대로 343 A동,자동차충전,35.225657,129.213988,2592.0,1069000.0,2.770848e+09,11.513517,0.458557,45
9,부산,기장군,한국에너지산업(주),부산 기장군 기장읍 기장대로413번길 38,용기+자동차+탱크+13kg용기,35.235543,129.215629,2473.0,1063000.0,2.628799e+09,10.436889,0.829890,241


In [223]:
LPG_group = df_LPG.groupby('행정구').count()
LPG_group.describe()

,행정시,업소명,소재지,관리구분,위도,경도,면적,공시지가,토지비용,주변충전소 최단거리,주변고속도로 최단거리,교차로 수
count,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,3.100000,3.100000,3.100000,3.100000,3.100000,3.100000,3.100000,3.100000,3.100000,3.100000,3.100000,3.100000
std,2.233582,2.233582,2.233582,2.233582,2.233582,2.233582,2.233582,2.233582,2.233582,2.233582,2.233582,2.233582
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
50%,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000
75%,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
max,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000


#### LPG 충전소 수 적합도

In [224]:
LPG_pro,LPG_pro_negative = cal_norm(LPG_group['업소명'].mean(),LPG_group['업소명'].std(),
LPG_group['업소명'].min(),LPG_group['업소명'].max(),LPG_standard,False)

### 전기차 충전소 현황 데이터 불러오기

In [225]:
EVCS_file_path = 'C:\\Users\\SAMSUNG\\Desktop\\Green-Car-Charging-Station-Site-Selection\\Data\\부산 전기차 충전소 현황.csv'
df_EVCS = pd.read_csv(EVCS_file_path, encoding='cp949')
df_EVCS

,운영기관,충전소,충전기ID,충전기타입,지역,시군구,주소,위도,경도
0,환경부(한국자동차환경협회),kbs 부산방송총국,1,DC차데모+AC3상+DC콤보,부산광역시,수영구,부산광역시 수영구 수영로 429,35.144806,129.107595
1,환경부(한국자동차환경협회),kbs 부산방송총국,11,DC콤보,부산광역시,수영구,부산광역시 수영구 수영로 429,35.144806,129.107595
2,환경부(한국자동차환경협회),kbs 부산방송총국,12,DC콤보,부산광역시,수영구,부산광역시 수영구 수영로 429,35.144806,129.107595
3,환경부(한국자동차환경협회),국립부산과학관,11,DC콤보,부산광역시,기장군,부산광역시 기장군 기장읍 석산리 293,35.204002,129.212987
4,환경부(한국자동차환경협회),국립부산과학관,12,DC콤보,부산광역시,기장군,부산광역시 기장군 기장읍 석산리 293,35.204002,129.212987
...,...,...,...,...,...,...,...,...,...
6291,한국전자금융,부산 플러스타워,1,AC완속,부산광역시,강서구,부산광역시 강서구 명지국제1로 56-1플러스타워(3598-2 플러스타워),35.093685,128.905372
6292,한국전자금융,부산 플러스타워,2,AC완속,부산광역시,강서구,부산광역시 강서구 명지국제1로 56-1플러스타워(3598-2 플러스타워),35.093685,128.905372
6293,한국전자금융,부산 휴마트괴정점,1,AC완속,부산광역시,사하구,부산광역시 사하구 낙동대로 123휴마트괴정점(괴정동 358-1),35.103200,129.002528
6294,한국전자금융,부산 휴마트괴정점,2,AC완속,부산광역시,사하구,부산광역시 사하구 낙동대로 123휴마트괴정점(괴정동 358-1),35.103200,129.002528


In [226]:
EVCS_group = df_EVCS.groupby('시군구').count()
EVCS_group

,운영기관,충전소,충전기ID,충전기타입,지역,주소,위도,경도
시군구,,,,,,,,
강서구,465,465,465,465,465,465,465,465
금정구,311,311,311,311,311,311,311,311
기장군,504,504,504,504,504,504,504,504
남구,681,681,681,681,681,681,681,681
동구,119,119,119,119,119,119,119,119
동래구,471,471,471,471,471,471,471,471
부산진구,583,583,583,583,583,583,583,583
북구,353,353,353,353,353,353,353,353
사상구,315,315,315,315,315,315,315,315


#### 전기차 충전소 수 적합도

In [227]:
EVCS_group['충전소'] = EVCS_group['충전소'].astype(int)
EVCS_pro,EVCS_pro_negative = cal_norm(EVCS_group['충전소'].mean(),EVCS_group['충전소'].std(),
EVCS_group['충전소'].min(),EVCS_group['충전소'].max(),EVCS_standard,True)

### 수소충전소 현황 데이터 불러오기

In [228]:
HVCS_file_path = 'C:\\Users\\SAMSUNG\\Desktop\\Green-Car-Charging-Station-Site-Selection\\Data\\부산 수소차 충전소 현황.csv'
df_HVCS = pd.read_csv(HVCS_file_path, encoding='cp949')
df_HVCS

,충전소,충전기,주소,위도,경도
0,상암수소충전소,1,서울특별시 마포구 하늘공원로 108,37.569685,126.880073
1,마곡에코수소충전소,1,서울특별시 강서구 양천로 225,37.573016,126.829208
2,오곡수소충전소,1,서울특별시 강서구 벌말로 271,37.555845,126.768181
3,H국회수소충전소,1,서울특별시 영등포구 의사당대로 1,37.532912,126.916404
4,양재수소충전소,1,서울특별시 서초구 바우뫼로12길 73,37.468378,127.034374
...,...,...,...,...,...
105,하이넷 통영용남수소충전소,1,경상남도 통영시 용남면 남해안대로 38,34.891246,128.471819
106,김해수소충전소,2,경상남도 김해시 김해대로2596번길 53,0.000000,0.000000
107,하이넷 거제장평수소충전소,1,경상남도 거제시 거제대로 5050,34.894764,128.587447
108,양산시증산수소충전소,1,경상남도 양산시 물금읍 메기로 114,35.303404,129.010257


In [229]:
address_city = []
address_gu = []
for i in range(df_HVCS.shape[0]):
    strings = df_HVCS.iloc[i]['주소'].split()
    address_city.append(strings.pop(0))
    address_gu.append(strings.pop(0))
df_HVCS.insert(2,"시",address_city)
df_HVCS.insert(3,"구",address_gu)    
df_HVCS

,충전소,충전기,시,구,주소,위도,경도
0,상암수소충전소,1,서울특별시,마포구,서울특별시 마포구 하늘공원로 108,37.569685,126.880073
1,마곡에코수소충전소,1,서울특별시,강서구,서울특별시 강서구 양천로 225,37.573016,126.829208
2,오곡수소충전소,1,서울특별시,강서구,서울특별시 강서구 벌말로 271,37.555845,126.768181
3,H국회수소충전소,1,서울특별시,영등포구,서울특별시 영등포구 의사당대로 1,37.532912,126.916404
4,양재수소충전소,1,서울특별시,서초구,서울특별시 서초구 바우뫼로12길 73,37.468378,127.034374
...,...,...,...,...,...,...,...
105,하이넷 통영용남수소충전소,1,경상남도,통영시,경상남도 통영시 용남면 남해안대로 38,34.891246,128.471819
106,김해수소충전소,2,경상남도,김해시,경상남도 김해시 김해대로2596번길 53,0.000000,0.000000
107,하이넷 거제장평수소충전소,1,경상남도,거제시,경상남도 거제시 거제대로 5050,34.894764,128.587447
108,양산시증산수소충전소,1,경상남도,양산시,경상남도 양산시 물금읍 메기로 114,35.303404,129.010257


In [230]:
df_HVCS_group = df_HVCS.groupby('시').count()
df_HVCS_group

,충전소,충전기,구,주소,위도,경도
시,,,,,,
강원도,7,7,7,7,7,7
경기도,19,19,19,19,19,19
경상남도,12,12,12,12,12,12
경상북도,2,2,2,2,2,2
광주광역시,6,6,6,6,6,6
대구광역시,4,4,4,4,4,4
대전광역시,7,7,7,7,7,7
부산광역시,3,3,3,3,3,3
서울특별시,6,6,6,6,6,6


In [231]:
df_busan_HVCS = df_HVCS[df_HVCS['시']=='부산광역시'].reset_index(drop=True)
df_busan_HVCS

,충전소,충전기,시,구,주소,위도,경도
0,서부산NK수소충전소,1,부산광역시,강서구,부산광역시 강서구 가락대로 347,35.113171,128.843291
1,H부산수소충전소,3,부산광역시,사상구,부산광역시 사상구 학감대로 167,35.145864,128.987651
2,하이넷부산정관수소충,1,부산광역시,기장군,부산광역시 기장군 정관읍 신정관로 187,35.328714,129.201710


#### 수소차 충전소 수 적합도

In [232]:
HVCS_pro,HVCS_pro_negative = cal_norm(df_HVCS_group['충전소'].mean(),df_HVCS_group['충전소'].std(),
df_HVCS_group['충전소'].min(),df_HVCS_group['충전소'].max(),HVCS_standard,True)
HVCS_pro

0.9583233017989948

### 교통 편의성 데이터

In [233]:
# 주차장 정보
parking_intersection_file_path = 'C:\\Users\\SAMSUNG\\Desktop\\Green-Car-Charging-Station-Site-Selection\\Data\\부산 주차장 현황(교차로).csv'
df_parking_intersection = pd.read_csv(parking_intersection_file_path, encoding='cp949')
df_parking_intersection

,운영형태,행정구,관리기관명,주차장명,주차장관리번호,소재지도로명주소,주차장구분,주차장유형,주차구획수,위도,경도,교차로,주변고속도로까지 최단거리,주변충전소 최단거리,환경적요인,사회적요인,경제적요인,기술적요인,최종확률
0,구(군)직영,사상구,부산광역시 사상구청,감전동주변,2010000013,사상구 감전동 589-2,공영,노외,26,35.154108,128.980071,239,4346.343726,0.061228,0.838573,0.643981,0.252453,0.488554,0.532397
1,민간위탁,사상구,부산광역시 사상구청,엄궁초등학교지하주차장,2010000016,사상구 엄궁로191번길 23,공영,노외,93,35.128446,128.973403,85,7240.940505,0.112529,0.838573,0.493981,0.252453,0.488554,0.472397
2,구(군)직영,사상구,부산광역시 사상구청,일신주택주변,2011000003,사상구 주례3동 545-3,공영,노외,10,35.145207,129.000672,165,5584.115172,0.247823,0.838573,0.643981,0.252453,0.488554,0.532397
3,구(군)직영,사상구,부산광역시 사상구청,주례중학교주변,2012000004,사상구 주례2동 82-13,공영,노외,8,35.154384,129.011131,276,5040.598424,0.094006,0.838573,0.643981,0.252453,0.488554,0.532397
4,민간위탁,사상구,부산광역시 사상구청,모라1동행정복지센터주변,2007001725,사상구 백양대로 949-6,공영,노상,156,35.171044,128.988085,249,2512.034932,0.190308,0.838573,0.643981,0.752453,0.488554,0.682397
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
468,민간위탁,중구,부산광역시 중구청,중앙동 노외공영주차장,2009000002,중구 대청로135번길 36,공영,노외,29,35.104229,129.034810,593,10976.446670,0.143217,0.838573,0.443981,0.252453,0.393172,0.438090
469,민간운영,기장군,부산광역시 기장군청,1구역 2소,2008000002,기장군 기장읍 280-2,공영,노상,12,35.222141,129.206947,19,162.104257,1.603011,0.838573,0.493981,0.252453,0.327783,0.448282
470,민간운영,기장군,부산광역시 기장군청,1구역 3소,2008000003,기장군 기장읍 444-1,공영,노상,13,35.222141,129.206947,19,162.104257,1.603011,0.838573,0.493981,0.252453,0.327783,0.448282
471,민간운영,기장군,부산광역시 기장군청,3구역 1소,2008000006,기장군 기장읍 418-12,공영,노상,24,35.222141,129.206947,19,162.104257,1.603011,0.838573,0.493981,0.252453,0.327783,0.448282


#### 부산시 교차로수 적합도(전기)

In [234]:
intersection_E_pro, intersection_E_pro_negative = cal_norm(df_parking_intersection['교차로'].mean(),
                                                       df_parking_intersection['교차로'].std(),
                                                       df_parking_intersection['교차로'].min(),
                                                       df_parking_intersection['교차로'].max(),
                                                       intersection_standard,
                                                       False)

#### 부산시 교차로수 적합도(수소)

In [235]:
df_LPG

,행정시,행정구,업소명,소재지,관리구분,위도,경도,면적,공시지가,토지비용,주변충전소 최단거리,주변고속도로 최단거리,교차로 수
0,부산,강서구,개인택시신공항충전소,부산 강서구 경전철로188번길 95,자동차충전,35.198669,128.966594,2545.0,2144000.0,5.456480e+09,6.175720,0.815596,52
1,부산,강서구,부산공항LPG충전소,부산 강서구 공항로 459,자동차충전,35.149064,128.951891,3105.0,1174000.0,3.645270e+09,3.270797,5.631472,16
2,부산,강서구,신항만에너지,부산 강서구 낙동남로 507,자동차충전,35.112947,128.875260,3105.0,1174000.0,3.645270e+09,2.907968,11.968669,76
3,부산,강서구,르노삼성자동차(주),부산 강서구 르노삼성대로 61,자동차충전,35.095408,128.883215,1538.0,640100.0,9.844738e+08,4.134222,13.975751,82
4,부산,강서구,죽림LPG충전소,부산 강서구 호계로 102,자동차충전,35.200736,128.893462,1580.0,1017000.0,1.606860e+09,10.512855,2.696444,16
5,부산,강서구,가락충전소,부산 강서구 호계로 195,자동차충전,35.208847,128.890148,2600.0,1017000.0,2.644200e+09,11.295108,1.747535,20
6,부산,금정구,(주)해연,부산 금정구 개좌로 225-15,용기+탱크+13kg용기,35.230796,129.122388,3346.0,1370000.0,4.584020e+09,12.992938,3.145687,119
7,부산,금정구,(주)신진가스,부산 금정구 반송로 433,자동차충전,35.215006,129.111143,1646.4,2797000.0,4.604981e+09,13.435925,3.870264,407
8,부산,기장군,마라톤가스(주)청강충전소,부산 기장군 기장읍 기장대로 343 A동,자동차충전,35.225657,129.213988,2592.0,1069000.0,2.770848e+09,11.513517,0.458557,45
9,부산,기장군,한국에너지산업(주),부산 기장군 기장읍 기장대로413번길 38,용기+자동차+탱크+13kg용기,35.235543,129.215629,2473.0,1063000.0,2.628799e+09,10.436889,0.829890,241


In [236]:
intersection_H_pro, intersection_H_pro_negative = cal_norm(df_LPG['교차로 수'].mean(),
                                                            df_LPG['교차로 수'].std(),
                                                            df_LPG['교차로 수'].min(),
                                                            df_LPG['교차로 수'].max(),
                                                            intersection_standard,
                                                            False)

### 주변 충전소와의 거리 데이터(전기,수소)

In [237]:
parking_EV_path = 'C:\\Users\\SAMSUNG\\Desktop\\Green-Car-Charging-Station-Site-Selection\\Data\\부산 주차장 현황(교차로).csv'
df_parking_EV = pd.read_csv(parking_EV_path, encoding='cp949')
df_parking_EV

,운영형태,행정구,관리기관명,주차장명,주차장관리번호,소재지도로명주소,주차장구분,주차장유형,주차구획수,위도,경도,교차로,주변고속도로까지 최단거리,주변충전소 최단거리,환경적요인,사회적요인,경제적요인,기술적요인,최종확률
0,구(군)직영,사상구,부산광역시 사상구청,감전동주변,2010000013,사상구 감전동 589-2,공영,노외,26,35.154108,128.980071,239,4346.343726,0.061228,0.838573,0.643981,0.252453,0.488554,0.532397
1,민간위탁,사상구,부산광역시 사상구청,엄궁초등학교지하주차장,2010000016,사상구 엄궁로191번길 23,공영,노외,93,35.128446,128.973403,85,7240.940505,0.112529,0.838573,0.493981,0.252453,0.488554,0.472397
2,구(군)직영,사상구,부산광역시 사상구청,일신주택주변,2011000003,사상구 주례3동 545-3,공영,노외,10,35.145207,129.000672,165,5584.115172,0.247823,0.838573,0.643981,0.252453,0.488554,0.532397
3,구(군)직영,사상구,부산광역시 사상구청,주례중학교주변,2012000004,사상구 주례2동 82-13,공영,노외,8,35.154384,129.011131,276,5040.598424,0.094006,0.838573,0.643981,0.252453,0.488554,0.532397
4,민간위탁,사상구,부산광역시 사상구청,모라1동행정복지센터주변,2007001725,사상구 백양대로 949-6,공영,노상,156,35.171044,128.988085,249,2512.034932,0.190308,0.838573,0.643981,0.752453,0.488554,0.682397
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
468,민간위탁,중구,부산광역시 중구청,중앙동 노외공영주차장,2009000002,중구 대청로135번길 36,공영,노외,29,35.104229,129.034810,593,10976.446670,0.143217,0.838573,0.443981,0.252453,0.393172,0.438090
469,민간운영,기장군,부산광역시 기장군청,1구역 2소,2008000002,기장군 기장읍 280-2,공영,노상,12,35.222141,129.206947,19,162.104257,1.603011,0.838573,0.493981,0.252453,0.327783,0.448282
470,민간운영,기장군,부산광역시 기장군청,1구역 3소,2008000003,기장군 기장읍 444-1,공영,노상,13,35.222141,129.206947,19,162.104257,1.603011,0.838573,0.493981,0.252453,0.327783,0.448282
471,민간운영,기장군,부산광역시 기장군청,3구역 1소,2008000006,기장군 기장읍 418-12,공영,노상,24,35.222141,129.206947,19,162.104257,1.603011,0.838573,0.493981,0.252453,0.327783,0.448282


#### 부산의 주변충전소 최단거리 적합성(전기)

In [238]:
EV_distance_pro, EV_distance_pro_negative = cal_norm(df_parking_EV['주변충전소 최단거리'].mean(),
                                                    df_parking_EV['주변충전소 최단거리'].std(),
                                                    df_parking_EV['주변충전소 최단거리'].min(),
                                                    df_parking_EV['주변충전소 최단거리'].max(),
                                                    EV_distance_standard,
                                                    False)

#### 부산의 주변충전소 최단거리(수소)

In [239]:
LPG_file_path = 'C:\\Users\\SAMSUNG\\Desktop\\Green-Car-Charging-Station-Site-Selection\\Data\\부산 LPG 충전소 현황(한국가스안전공사).csv'
df_LPG = pd.read_csv(LPG_file_path, encoding='cp949')
df_LPG

,행정시,행정구,업소명,소재지,관리구분,위도,경도,면적,공시지가,토지비용,주변충전소 최단거리,주변고속도로 최단거리,교차로 수
0,부산,강서구,개인택시신공항충전소,부산 강서구 경전철로188번길 95,자동차충전,35.198669,128.966594,2545.0,2144000.0,5.456480e+09,6.175720,0.815596,52
1,부산,강서구,부산공항LPG충전소,부산 강서구 공항로 459,자동차충전,35.149064,128.951891,3105.0,1174000.0,3.645270e+09,3.270797,5.631472,16
2,부산,강서구,신항만에너지,부산 강서구 낙동남로 507,자동차충전,35.112947,128.875260,3105.0,1174000.0,3.645270e+09,2.907968,11.968669,76
3,부산,강서구,(주)지원네트웍스 부산강서LPG충전소,부산 강서구 낙동북로 310,자동차충전,35.212507,128.962572,1396.0,1140000.0,1.591440e+09,7.752939,0.864172,32
4,부산,강서구,신명지에너지,부산 강서구 르노삼성대로 560,자동차충전,35.097554,128.927351,481.0,700300.0,3.368443e+08,7.676891,11.748654,159
...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,부산,연제구,부산개인택시엘피지충전소,부산 연제구 월드컵대로 399,자동차충전,35.186013,129.055974,9212.0,688600.0,6.343383e+09,7.648635,5.612178,146
59,부산,영도구,(주)태종에너지,부산 영도구 영선대로 41,자동차충전,35.084996,129.040033,1610.2,2679000.0,4.313726e+09,8.277014,13.118878,255
60,부산,영도구,부산에너지,부산 영도구 해양로 160,용기+자동차+탱크+13kg용기,35.091275,129.069484,5801.0,989400.0,5.739509e+09,9.604391,13.821871,67
61,부산,영도구,개인택시 영도LPG충전소,부산 영도구 해양로 296,자동차충전,35.082176,129.076429,1818.0,349800.0,6.359364e+08,10.740567,14.353595,98


In [240]:
HV_distance_pro, HV_distance_pro_negative = cal_norm(df_LPG['주변충전소 최단거리'].mean(),
                                                    df_LPG['주변충전소 최단거리'].std(),
                                                    df_LPG['주변충전소 최단거리'].min(),
                                                    df_LPG['주변충전소 최단거리'].max(),
                                                    HV_distance_standard,
                                                    False)

In [241]:
# from haversine import haversine
# result = []
# for L_lat,L_lon in LPG_location:
#     min = 100000000000000
#     place_LPG = (L_lat,L_lon)
#     for H_lat,H_lon in HVCS_location:
#         place_HVCS = (H_lat,H_lon)
#         distance = haversine(place_LPG,place_HVCS)
#         if min > distance:
#             min = distance
#     result.append(min)
# print(result)

### 주변 고속도로 여부(정성적)
  
    고속도로인 경우 적합 : 1.0, 부적합 : 0.0 
    고속도로가 아닌 경우 적합 : 0.0, 부적합 : 1.0

In [242]:
highway_file_path = 'C:\\Users\\SAMSUNG\\Desktop\\Green-Car-Charging-Station-Site-Selection\\Data\\고속도로 좌표(한국도로공사).csv'
df_highway = pd.read_csv(highway_file_path, encoding='cp949')
df_highway

,노선번호,도로명,이정,X좌표값,Y좌표값,GRS80X좌표값,GRS80Y좌표값
0,10,경부선,0.0,35.246578,129.093207,390524.2902,296459.9426
1,10,경부선,0.1,35.247249,129.093941,390589.5768,296535.7480
2,10,경부선,0.2,35.247919,129.094676,390654.8636,296611.5534
3,10,경부선,0.3,35.248590,129.095410,390720.1503,296687.3588
4,10,경부선,0.4,35.249260,129.096145,390785.4371,296763.1641
...,...,...,...,...,...,...,...
48236,6000,부산외곽순환선,48.4,35.282415,129.213261,401364.7093,300673.9867
48237,6000,부산외곽순환선,48.5,35.282408,129.214360,401464.7332,300675.4880
48238,6000,부산외곽순환선,48.6,35.282349,129.215457,401564.6723,300671.1065
48239,6000,부산외곽순환선,48.7,35.282236,129.216547,401664.1809,300660.8570


In [243]:
# 부산 고속도로 추출
busan_highway = ['경부선','남해선(순천부산)','중앙선','동해선','부산포항선','부산외곽순환선']
df_highway = df_highway[df_highway['도로명'].isin(busan_highway)].reset_index(drop=True)
df_highway 

,노선번호,도로명,이정,X좌표값,Y좌표값,GRS80X좌표값,GRS80Y좌표값
0,10,경부선,0.0,35.246578,129.093207,390524.2902,296459.9426
1,10,경부선,0.1,35.247249,129.093941,390589.5768,296535.7480
2,10,경부선,0.2,35.247919,129.094676,390654.8636,296611.5534
3,10,경부선,0.3,35.248590,129.095410,390720.1503,296687.3588
4,10,경부선,0.4,35.249260,129.096145,390785.4371,296763.1641
...,...,...,...,...,...,...,...
11687,6000,부산외곽순환선,48.4,35.282415,129.213261,401364.7093,300673.9867
11688,6000,부산외곽순환선,48.5,35.282408,129.214360,401464.7332,300675.4880
11689,6000,부산외곽순환선,48.6,35.282349,129.215457,401564.6723,300671.1065
11690,6000,부산외곽순환선,48.7,35.282236,129.216547,401664.1809,300660.8570


In [244]:
h_lat = df_highway['X좌표값'].tolist()
h_lon = df_highway['Y좌표값'].tolist()
h_loc = list(zip(h_lat,h_lon))

In [245]:
df_LPG

,행정시,행정구,업소명,소재지,관리구분,위도,경도,면적,공시지가,토지비용,주변충전소 최단거리,주변고속도로 최단거리,교차로 수
0,부산,강서구,개인택시신공항충전소,부산 강서구 경전철로188번길 95,자동차충전,35.198669,128.966594,2545.0,2144000.0,5.456480e+09,6.175720,0.815596,52
1,부산,강서구,부산공항LPG충전소,부산 강서구 공항로 459,자동차충전,35.149064,128.951891,3105.0,1174000.0,3.645270e+09,3.270797,5.631472,16
2,부산,강서구,신항만에너지,부산 강서구 낙동남로 507,자동차충전,35.112947,128.875260,3105.0,1174000.0,3.645270e+09,2.907968,11.968669,76
3,부산,강서구,(주)지원네트웍스 부산강서LPG충전소,부산 강서구 낙동북로 310,자동차충전,35.212507,128.962572,1396.0,1140000.0,1.591440e+09,7.752939,0.864172,32
4,부산,강서구,신명지에너지,부산 강서구 르노삼성대로 560,자동차충전,35.097554,128.927351,481.0,700300.0,3.368443e+08,7.676891,11.748654,159
...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,부산,연제구,부산개인택시엘피지충전소,부산 연제구 월드컵대로 399,자동차충전,35.186013,129.055974,9212.0,688600.0,6.343383e+09,7.648635,5.612178,146
59,부산,영도구,(주)태종에너지,부산 영도구 영선대로 41,자동차충전,35.084996,129.040033,1610.2,2679000.0,4.313726e+09,8.277014,13.118878,255
60,부산,영도구,부산에너지,부산 영도구 해양로 160,용기+자동차+탱크+13kg용기,35.091275,129.069484,5801.0,989400.0,5.739509e+09,9.604391,13.821871,67
61,부산,영도구,개인택시 영도LPG충전소,부산 영도구 해양로 296,자동차충전,35.082176,129.076429,1818.0,349800.0,6.359364e+08,10.740567,14.353595,98


In [246]:
# 주차장과 주변 고속도로 최단거리 계산 후 column 추가
parking_file_path = 'C:\\Users\\SAMSUNG\\Desktop\\Green-Car-Charging-Station-Site-Selection\\Data\\부산 주차장 현황.csv'
df_parking = pd.read_csv(parking_file_path, encoding='cp949')
df_parking

,운영형태,행정구,관리기관명,주차장명,주차장관리번호,소재지도로명주소,주차장구분,주차장유형,주차구획수,위도,경도,주변충전소 최단거리,주변고속도로까지 최단거리
0,구(군)직영,사상구,부산광역시 사상구청,감전동주변,2010000013,사상구 감전동 589-2,공영,노외,26,35.154108,128.980071,0.061228,4346.343726
1,구(군)직영,사상구,부산광역시 사상구청,장성주차장,2010000015,사상구 주례2동 92-278,공영,노외,16,0.000000,0.000000,0.000000,-
2,민간위탁,사상구,부산광역시 사상구청,엄궁초등학교지하주차장,2010000016,사상구 엄궁로191번길 23,공영,노외,93,35.128446,128.973403,0.112529,7240.940505
3,구(군)직영,사상구,부산광역시 사상구청,일신주택주변,2011000003,사상구 주례3동 545-3,공영,노외,10,35.145207,129.000672,0.247823,5584.115172
4,구(군)직영,사상구,부산광역시 사상구청,주례중학교주변,2012000004,사상구 주례2동 82-13,공영,노외,8,35.154384,129.011131,0.094006,5040.598424
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,구(군)직영,기장군,부산광역시 기장군청,정관읍 상업지역 노상주차장,2016000017,기장군 정관면 732,공영,노상,46,0.000000,0.000000,0.000000,-
610,민간운영,기장군,부산광역시 기장군청,1구역 2소,2008000002,기장군 기장읍 280-2,공영,노상,12,35.222141,129.206947,1.603011,162.1042569
611,민간운영,기장군,부산광역시 기장군청,1구역 3소,2008000003,기장군 기장읍 444-1,공영,노상,13,35.222141,129.206947,1.603011,162.1042569
612,민간운영,기장군,부산광역시 기장군청,3구역 1소,2008000006,기장군 기장읍 418-12,공영,노상,24,35.222141,129.206947,1.603011,162.1042569


In [247]:
df_parking_highway = df_parking[df_parking['주변고속도로까지 최단거리']!='-'].reset_index(drop=True)
df_parking_highway['주변고속도로까지 최단거리'] = df_parking_highway['주변고속도로까지 최단거리'].astype(float)

#### 부산의 고속도로 근접 적합성(전기)

In [248]:
highway_E_pro, highway_E_pro_negative = cal_norm(df_parking_highway['주변고속도로까지 최단거리'].mean(),
                                                 df_parking_highway['주변고속도로까지 최단거리'].std(),
                                                 df_parking_highway['주변고속도로까지 최단거리'].min(),
                                                 df_parking_highway['주변고속도로까지 최단거리'].max(),
                                                 highway_standard_E,
                                                 True)

#### 부산의 고속도로 근접 적합성(수소)

In [249]:
highway_H_pro, highway_H_pro_negative = cal_norm(df_LPG['주변고속도로 최단거리'].mean(),
                                                 df_LPG['주변고속도로 최단거리'].std(),
                                                 df_LPG['주변고속도로 최단거리'].min(),
                                                 df_LPG['주변고속도로 최단거리'].max(),
                                                 highway_standard_H,
                                                 True)

## 상위 요소 - 경제적 요소

### 전기차 충전기 가격 불러오기(전기차)

In [250]:
EVCS_cost_file_path = 'C:\\Users\\SAMSUNG\\Desktop\\Green-Car-Charging-Station-Site-Selection\\Data\\전기차 충전기 가격.csv'
df_EVCS_cost = pd.read_csv(EVCS_cost_file_path, encoding='cp949')
df_EVCS_cost


,모델명,타입,충전기가격,전격용량,충전기
0,JC-6511PS-B-PO-BC,벽부착형 또는 스탠드형(폴 포함),1800000,7 kW,완속
1,JEV-AW-107,벽부착형,2000000,7 kWh,완속
2,JEV-AS-107,스탠드형,2000000,7 kWh,완속
3,CS 100X 2BC01W,벽부착형,1452080,7 kWh,완속
4,CS 500A 2BC04W,벽부착형,1889248,7 kWh,완속
5,CS 200A 2BC03C,스탠드형,2018074,7 kWh,완속
6,CS 200A 3BC01C,스탠드형,3435449,14 kWh,완속
7,UK-NC7W-TCE,벽부착형 또는 스탠드형(폴 포함),1800000,7 kW,완속
8,PECOH-HSC,벽부착형 또는 스탠드형(폴 포함),770000,7 kW,완속
9,SSEVC21F-BCCI100KW,스탠드형,58000000,100kW,급속


In [251]:
EVCS_cost, EVCS_cost_negative = cal_norm(df_EVCS_cost['충전기가격'].mean(),df_EVCS_cost['충전기가격'].std(),
df_EVCS_cost['충전기가격'].min(),df_EVCS_cost['충전기가격'].max(),EVCS_cost_standard,True)

### 충전소 설치 비용 데이터 불러오기 (수소차) 
(250kg급 기준)

 파이프라인 방식 24억원

 튜브트레일러방식 22억원
 
 이동식 18억원
 
 천연가스개질방식은 46억원
 
 수전해방식은 31억원

천연가스 개질방식이 가장 비싸고, 이동식이 가장 저렴

In [252]:
HVCS_construction_cost_file_path = 'C:\\Users\\SAMSUNG\\Desktop\\Green-Car-Charging-Station-Site-Selection\\Data\\수소 충전소 종류별 구축 비용.csv'
df_HVCS_construction_cost = pd.read_csv(HVCS_construction_cost_file_path, encoding='cp949')
df_HVCS_construction_cost = df_HVCS_construction_cost.set_index('충전소 종류')
df_HVCS_construction_cost


,구축 비용
충전소 종류,
파이프라인,2400000000
튜브트레일러방식,2200000000
이동식,1800000000
천연가스개질방식,4600000000
수전해방식,3100000000


#### 수소 충전소 구축 비용 적합도

In [253]:
HVCS_construction_cost_pro,HVCS_construction_cost_pro_negative  = cal_norm(df_HVCS_construction_cost['구축 비용'].mean(),df_HVCS_construction_cost['구축 비용'].std(),
df_HVCS_construction_cost['구축 비용'].min(),df_HVCS_construction_cost['구축 비용'].max(),HVCS_construction_cost_standard,True)
HVCS_construction_cost_pro

0.5379381229554602

### LPG 충전소 토지비용
- LPG 충전소의 면적과 공시지가를 통해 토지비용 도출

#### LPG 충전소 토지비용 적합도

In [254]:
LPG_land_pro,LPG_land_pro_negative = cal_norm(df_LPG['토지비용'].mean(),df_LPG['토지비용'].std(),
df_LPG['토지비용'].min(),df_LPG['토지비용'].max(),LPG_land_standard,True)
LPG_land_pro

0.8809981599397956

### 주차 구획 수 데이터 불러오기

In [255]:
parking_file_path = 'C:\\Users\\SAMSUNG\\Desktop\\Green-Car-Charging-Station-Site-Selection\\Data\\부산 주차장 현황(교차로).csv'
df_parking = pd.read_csv(parking_file_path, encoding='cp949')
df_parking

,운영형태,행정구,관리기관명,주차장명,주차장관리번호,소재지도로명주소,주차장구분,주차장유형,주차구획수,위도,경도,교차로,주변고속도로까지 최단거리,주변충전소 최단거리,환경적요인,사회적요인,경제적요인,기술적요인,최종확률
0,구(군)직영,사상구,부산광역시 사상구청,감전동주변,2010000013,사상구 감전동 589-2,공영,노외,26,35.154108,128.980071,239,4346.343726,0.061228,0.838573,0.643981,0.252453,0.488554,0.532397
1,민간위탁,사상구,부산광역시 사상구청,엄궁초등학교지하주차장,2010000016,사상구 엄궁로191번길 23,공영,노외,93,35.128446,128.973403,85,7240.940505,0.112529,0.838573,0.493981,0.252453,0.488554,0.472397
2,구(군)직영,사상구,부산광역시 사상구청,일신주택주변,2011000003,사상구 주례3동 545-3,공영,노외,10,35.145207,129.000672,165,5584.115172,0.247823,0.838573,0.643981,0.252453,0.488554,0.532397
3,구(군)직영,사상구,부산광역시 사상구청,주례중학교주변,2012000004,사상구 주례2동 82-13,공영,노외,8,35.154384,129.011131,276,5040.598424,0.094006,0.838573,0.643981,0.252453,0.488554,0.532397
4,민간위탁,사상구,부산광역시 사상구청,모라1동행정복지센터주변,2007001725,사상구 백양대로 949-6,공영,노상,156,35.171044,128.988085,249,2512.034932,0.190308,0.838573,0.643981,0.752453,0.488554,0.682397
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
468,민간위탁,중구,부산광역시 중구청,중앙동 노외공영주차장,2009000002,중구 대청로135번길 36,공영,노외,29,35.104229,129.034810,593,10976.446670,0.143217,0.838573,0.443981,0.252453,0.393172,0.438090
469,민간운영,기장군,부산광역시 기장군청,1구역 2소,2008000002,기장군 기장읍 280-2,공영,노상,12,35.222141,129.206947,19,162.104257,1.603011,0.838573,0.493981,0.252453,0.327783,0.448282
470,민간운영,기장군,부산광역시 기장군청,1구역 3소,2008000003,기장군 기장읍 444-1,공영,노상,13,35.222141,129.206947,19,162.104257,1.603011,0.838573,0.493981,0.252453,0.327783,0.448282
471,민간운영,기장군,부산광역시 기장군청,3구역 1소,2008000006,기장군 기장읍 418-12,공영,노상,24,35.222141,129.206947,19,162.104257,1.603011,0.838573,0.493981,0.252453,0.327783,0.448282


#### 주차 구획수 통계 구하기

In [256]:
df_parking_area = df_parking['주차구획수'].astype(int)
df_parking_area.describe()

count    473.000000
mean      61.232558
std       71.651440
min      -41.000000
25%       18.000000
50%       36.000000
75%       77.000000
max      414.000000
Name: 주차구획수, dtype: float64

#### 주차구획 수 적합도

In [257]:
parking_area_pro,parking_area_pro_negative = cal_norm(df_parking_area.mean(),df_parking_area.std(),df_parking_area.min(),df_parking_area.max(),parking_area_standard,False)
parking_area_pro

0.302234490619828

## 상위 요소 - 기술적 요소

### 복합 충전소 여부 (수소차)

기존의 LPG 충전소 부지에 수소충전소를 세우는 경우 "적합" 100%로 설정, 반대의 경우 "부적합" 100%로 설정함

In [258]:
# 기존 LPG 충전소 위경도 추출
LPG_lat = df_LPG['위도'].tolist()
LPG_lon = df_LPG['경도'].tolist()
location_LPG = list(zip(LPG_lat,LPG_lon))

In [259]:
# 기존 LPG 충전소가 입지인 경우
location_place = (35.18601331, 129.0559741) # 초기 입력좌표

if location_place in location_LPG:
    complex_pro = 1.0
else:
    complex_pro = 0.0


### 충전시간에 따른 충전방식 (전기차, 정성적) 
 유동인구가 해당지역의 거주인구보다 많은 지역에 해당하는 경우 급속충전의 이용율이 높음
 
 따라서 "급속충전" 100%로 설정, 반대의 경우는 "완속충전" 100%로 설정함  

In [260]:
# 유동인구와 고정인구에 따른 충전방식 적합도 계산 
def charge_type(people,floating):
    '''
    foottraffic_pro : 유동인구 적합도
    busan_people_pro : 고정인구 적합도
    charger_pro : 설정하고자 하는 충전기 타입의 적합도 비율 초기값(0~1)
    '''
    
    # 급속 충전기 초기 확률:0.38
    charger_pro = charger_pro_default
    p_pro,a = cal_norm(df_busan_people['총인구수 (명)'].mean(),df_busan_people['총인구수 (명)'].std(),df_busan_people['총인구수 (명)'].min(),df_busan_people['총인구수 (명)'].max(),people,True)
    f_pro,b = cal_norm(df_foottraffic['월 평균'].mean(),df_foottraffic['월 평균'].std(),df_foottraffic['월 평균'].min(),df_foottraffic['월 평균'].max(),floating,True)
    
    # 유동인구와 기존인구와 차이가 나는 만큼 더 많은 확률 부여
    if f_pro >= p_pro:
        diff = f_pro - p_pro
        charger_pro += diff
        if charger_pro > 1:
            charger_pro = 1.0
        
    elif f_pro < p_pro:
        diff = p_pro - f_pro
        charger_pro -= diff
        if charger_pro < 0:
            charger_pro = 0.0
        
    return charger_pro, 1-charger_pro

#### 부산의 전기차 충전소의 타입

In [261]:
charger_pro_fast = 0.38
charger_pro_standard = 0.62
print(charger_pro_fast, charger_pro_standard)

0.38 0.62


## 결합 확률로 상위 요인 도출하기


In [262]:
# E는 전기차, H는 수소차
variable_environment = [] # 환경변수는 전기/수소 동일
variable_social_E = []
variable_social_H = []
variable_economic_E = []
variable_economic_H = []
variable_technical_E = []
variable_technical_H = []

## 상위 요인 결합 가중치
weight_environment = [0.13,0.13,0.13,0.13,0.13,0.13,0.22]   # co, so2, no2, ozone, pm10, pm25, 통합대기환경지수(khai)
weight_social_E = [0.1,0.1,0.3,0.15,0.15,0.1,0.1]           # 고정인구, 유동인구, 전기차 수, 전기차 충전소 수, 교통 편의성, 고속도로 여부, 주변충전소 최단거리
weight_social_H = [0.1,0.1,0.1,0.2,0.15,0.15,0.1,0.1]       # 고정인구, 유동인구, LPG 충전소 수, 수소차 수, 수소차 충전소 수, 교통 편의성, 고속도로 여부, 주변충전소 최단거리
weight_economic_E = [0.5,0.5]                               # 전기차충전기 설치비용, 주차 구획수
weight_economic_H = [0.5,0.5]                               # LPG 충전소 토지비용, 수소충전소 구축 비용
weight_technical_E = [1.0]                                  # 급속/완속 적합성
weight_technical_H = [1.0]                                  # 복합 충전소 여부

## 최종 확률 결합 가중치
weight_total = [0.15, 0.4, 0.3, 0.15]     # 환경적 요인, 사회적 요인, 경제적 요인, 기술적 요인

variable_environment.append(co_pro)
variable_environment.append(so2_pro)
variable_environment.append(no2_pro)
variable_environment.append(o3_pro)
variable_environment.append(pm10_pro)
variable_environment.append(pm25_pro)
variable_environment.append(khai_pro)


variable_social_E.append(busan_people_pro)
variable_social_E.append(foottraffic_pro)
variable_social_E.append(elec_vehicle_pro)
variable_social_E.append(EVCS_pro)
variable_social_E.append(intersection_E_pro)
variable_social_E.append(EV_distance_pro)
variable_social_E.append(highway_E_pro)

variable_social_H.append(busan_people_pro)
variable_social_H.append(foottraffic_pro)
variable_social_H.append(LPG_pro)
variable_social_H.append(hidro_vehicle_pro)
variable_social_H.append(HVCS_pro)
variable_social_H.append(intersection_H_pro)
variable_social_H.append(HV_distance_pro)
variable_social_H.append(highway_H_pro)


variable_economic_E.append(EVCS_cost)
variable_economic_E.append(parking_area_pro)

variable_economic_H.append(LPG_land_pro)
variable_economic_H.append(HVCS_construction_cost_pro)


variable_technical_E.append(charger_pro_fast)

variable_technical_H.append(complex_pro)


print("환경변수(전기,수소)",variable_environment)
print("사회변수(전기)",variable_social_E)
print("사회변수(수소)",variable_social_H)
print("경제변수(전기)",variable_economic_E)
print("경제변수(수소)",variable_economic_H)
print("기술변수(전기)",variable_technical_E)
print("기술변수(수소)",variable_technical_H)

환경변수(전기,수소) [0.8287531282163694, 0.7911203783441816, 0.909590433996196, 0.8017202428389887, 0.8866811666335976, 0.8415688322677302, 0.8220286272881268]
사회변수(전기) [0.5383756609850885, 0.5848617448890694, 0.6626879332080854, 0.6344991485975281, 0.8271541215787344, 0.36267883177388427, 0.06454319041386954]
사회변수(수소) [0.5383756609850885, 0.5848617448890694, 0.7148108415065166, 0.4304353506049742, 0.9583233017989948, 0.5666707016477999, 0.6587994540723359, 0.2608072089192539]
경제변수(전기) [0.5049067756260079, 0.302234490619828]
경제변수(수소) [0.8809981599397956, 0.5379381229554602]
기술변수(전기) [0.38]
기술변수(수소) [1.0]


### 결합확률(Joint Probability) 함수

In [263]:
# 결합 확률 구하기
def joint_pro(variable, weight):
    result = 0.0
    variable_weight = list(zip(variable,weight))
    for pro, wei in variable_weight:
        result += pro*wei
    return result

In [264]:
# 상위 요소 결합 확률
x_joint_pro = round(joint_pro(variable_environment,weight_environment),6)
social_joint_pro_E = round(joint_pro(variable_social_E,weight_social_E),6)
social_joint_pro_H = round(joint_pro(variable_social_H,weight_social_H),6)
economic_joint_pro_E = round(joint_pro(variable_economic_E,weight_economic_E),6)
economic_joint_pro_H = round(joint_pro(variable_economic_H,weight_economic_H),6)
technical_joint_pro_E = round(joint_pro(variable_technical_E,weight_technical_E),6)
technical_joint_pro_H = round(joint_pro(variable_technical_H,weight_technical_H),6)

print("Joint Probability")
print(x_joint_pro, "/", round(1-x_joint_pro,6))
print(social_joint_pro_E,"/", round(1-social_joint_pro_E,6))
print(social_joint_pro_H,"/", round(1-social_joint_pro_H,6))
print(economic_joint_pro_E,"/", round(1-economic_joint_pro_E,6))
print(economic_joint_pro_H,"/", round(1-economic_joint_pro_H,6))
print(technical_joint_pro_E,"/", round(1-technical_joint_pro_E,6))
print(technical_joint_pro_H,"/", round(1-technical_joint_pro_H,6))

Joint Probability
0.838573 / 0.161427
0.5731 / 0.4269
0.590602 / 0.409398
0.403571 / 0.596429
0.709468 / 0.290532
0.38 / 0.62
1.0 / 0.0


### 조건부확률(Conditional Probability) 분포표

 - 환경적 요인

    오존, 아황산가스, 이산화질소, 일산화탄소, 미세먼지(pm10, pm25)

 - 사회적 요인
   
    고정인구수, 유동인구 수, 친환경 자동차 등록대수, LPG 충전소 수, 충전소 수(전기,수소), 교통 편의성, 주변 충전소와의 거리, 고속도로 여부

 - 경제적 요인
   
    전기차 충전기 설치 비용, 수소 충전소 구축 비용, LPG 충전소 토지비용, 총 유지비용, 주차 구획 수

 - 기술적 요소

    복합 충전소 여부(수소), 충전시간에 따른 급속/완속 충전 방식

In [265]:
# 조건부 확률 구하기
def conditional_pro(weight):
    '''
    weight : 결합할 변수의 가중치 값(list)
    '''
    # 조건부확률테이블의 길이는 2^(하위 변수)의 크기를 가짐
    table_len = 2**len(weight)
    # 결과 리스트 초기화
    result_True = [0.0]*table_len
    
    for i in range(len(weight)): # 3번
        for j in range(table_len): # 8번
            # 변수의 T,F 거짓값
            cycle = table_len/(2**(i+1))
            if int(j/cycle)%2 == 0:
                result_True[j] += weight[i]
                # 부동소수점 오차 제거
                result_True[j] = round(result_True[j],6)

    result_False = list(reversed(result_True))

    return result_True, result_False


In [266]:
# 상위 요인 조건부 확률
environment_pro, environment_pro_negative = conditional_pro(weight_environment)
social_E_pro, social_E_pro_negative = conditional_pro(weight_social_E)
social_H_pro, social_H_pro_negative = conditional_pro(weight_social_H)
economic_E_pro, economic_E_pro_negative = conditional_pro(weight_economic_E)
economic_H_pro, economic_H_pro_negative = conditional_pro(weight_economic_H)
technical_E_pro, technical_E_pro_negative = conditional_pro(weight_technical_E)
technical_H_pro, technical_H_pro_negative = conditional_pro(weight_technical_H)

print("환경적 요소 조건부 확률",'\n', environment_pro,'\n',environment_pro_negative)
print("사회적(전기) 요소 조건부 확률",'\n', social_E_pro,'\n',social_E_pro_negative)
print("사회적(수소) 요소 조건부 확률",'\n', social_H_pro,'\n',social_H_pro_negative)
print("경제적(전기) 요소 조건부 확률",'\n', economic_E_pro,'\n',economic_E_pro_negative)
print("경제적(수소) 요소 조건부 확률",'\n', economic_H_pro,'\n',economic_H_pro_negative)
print("기술적(전기) 요소 조건부 확률",'\n', technical_E_pro,'\n',technical_E_pro_negative)
print("기술적(수소) 요소 조건부 확률",'\n', technical_H_pro,'\n',technical_H_pro_negative)

환경적 요소 조건부 확률 
 [1.0, 0.78, 0.87, 0.65, 0.87, 0.65, 0.74, 0.52, 0.87, 0.65, 0.74, 0.52, 0.74, 0.52, 0.61, 0.39, 0.87, 0.65, 0.74, 0.52, 0.74, 0.52, 0.61, 0.39, 0.74, 0.52, 0.61, 0.39, 0.61, 0.39, 0.48, 0.26, 0.87, 0.65, 0.74, 0.52, 0.74, 0.52, 0.61, 0.39, 0.74, 0.52, 0.61, 0.39, 0.61, 0.39, 0.48, 0.26, 0.74, 0.52, 0.61, 0.39, 0.61, 0.39, 0.48, 0.26, 0.61, 0.39, 0.48, 0.26, 0.48, 0.26, 0.35, 0.13, 0.87, 0.65, 0.74, 0.52, 0.74, 0.52, 0.61, 0.39, 0.74, 0.52, 0.61, 0.39, 0.61, 0.39, 0.48, 0.26, 0.74, 0.52, 0.61, 0.39, 0.61, 0.39, 0.48, 0.26, 0.61, 0.39, 0.48, 0.26, 0.48, 0.26, 0.35, 0.13, 0.74, 0.52, 0.61, 0.39, 0.61, 0.39, 0.48, 0.26, 0.61, 0.39, 0.48, 0.26, 0.48, 0.26, 0.35, 0.13, 0.61, 0.39, 0.48, 0.26, 0.48, 0.26, 0.35, 0.13, 0.48, 0.26, 0.35, 0.13, 0.35, 0.13, 0.22, 0.0] 
 [0.0, 0.22, 0.13, 0.35, 0.13, 0.35, 0.26, 0.48, 0.13, 0.35, 0.26, 0.48, 0.26, 0.48, 0.39, 0.61, 0.13, 0.35, 0.26, 0.48, 0.26, 0.48, 0.39, 0.61, 0.26, 0.48, 0.39, 0.61, 0.39, 0.61, 0.52, 0.74, 0.13, 0.35, 0.26, 0.48,

## 베이지안 네트워크 모델 구성

In [267]:
from IPython.core.display import Image
from networkx.drawing.nx_pydot import to_pydot

from pgmpy.factors.discrete import TabularCPD
from pgmpy.models import BayesianModel

In [268]:
from pgmpy.inference import VariableElimination
from pgmpy.factors.discrete import DiscreteFactor
from pgmpy.models import FactorGraph

In [269]:
### 전기차 요인
# 환경적 요인 하위 변수 확률
o3 = [[o3_pro], [1-o3_pro]]
so2 = [[so2_pro], [1-so2_pro]]
no2 = [[no2_pro], [1-no2_pro]]
co = [[co_pro], [1-co_pro]]
pm10 = [[pm10_pro], [1-pm10_pro]]
pm25 = [[pm25_pro], [1-pm25_pro]]
khai = [[khai_pro], [1-khai_pro]]

# 사회적 요인 하위 변수 확률
people = [[busan_people_pro],[1-busan_people_pro]]
floating = [[foottraffic_pro],[1-foottraffic_pro]]
EV = [[elec_vehicle_pro],[1-elec_vehicle_pro]]
EVCS = [[EVCS_pro],[1-EVCS_pro]]
intersection_E = [[intersection_E_pro],[1-intersection_E_pro]]
distance_E = [[EV_distance_pro],[1-EV_distance_pro]]
highway_E = [[highway_E_pro],[1-highway_E_pro]]

# 경제적 요인 하위 변수 확률
cost_E = [[EVCS_cost],[1-EVCS_cost]]
area = [[parking_area_pro],[1-parking_area_pro]]

# 기술적 요인 하위 변수 확률
type = [[charger_pro_fast],[1-charger_pro_fast]]


### 수소차 요인
# 환경적 요인은 전기차와 동일 -> 환경적 모델 하나로 구성

# 사회적 요인 하위 변수 확률
people = [[busan_people_pro],[1-busan_people_pro]]
floating = [[foottraffic_pro],[1-foottraffic_pro]]
LPG = [[LPG_pro],[1-LPG_pro]]
HV = [[hidro_vehicle_pro],[1-hidro_vehicle_pro]]
HVCS = [[HVCS_pro],[1-HVCS_pro]]
intersection_H = [[intersection_H_pro],[1-intersection_H_pro]]
distance_H = [[HV_distance_pro],[1-HV_distance_pro]]
highway_H = [[highway_H_pro],[1-highway_H_pro]]

# 경제적 요인 하위 변수 확률
cost_H = [[HVCS_construction_cost_pro],[1-HVCS_construction_cost_pro]]
LPG_land = [[LPG_land_pro],[1-LPG_land_pro]]

# 기술적 요인 하위 변수 확률
complex = [[complex_pro],[1-complex_pro]]

environment_model = BayesianModel([('o3', 'environment'), ('so2', 'environment'), ('no2', 'environment'), ('co', 'environment'),
                                   ('pm10', 'environment'), ('pm25', 'environment'), ('khai', 'environment')])
social_model_E = BayesianModel([('people','social_E'),('floating','social_E'),('EV','social_E'),('EVCS','social_E'),
                                ('intersection_E','social_E'),('distance_E','social_E'),('highway_E','social_E')])
social_model_H = BayesianModel([('people','social_H'),('floating','social_H'),('LPG','social_H'),('HV','social_H'),('HVCS','social_H'),
                                ('intersection_H','social_H'),('distance_H','social_H'),('highway_H','social_H')])
economic_model_E = BayesianModel([('cost_E','economic_E'),('area','economic_E')])
economic_model_H = BayesianModel([('cost_H','economic_H'),('LPG_land','economic_H')])
technical_model_E = BayesianModel([('type','technical_E')])
technical_model_H = BayesianModel([('complex','technical_H')])


### 환경적 요인의 하위 변수 모델링 
P_o3 = TabularCPD('o3', 2,values=o3)
P_so2 = TabularCPD('so2', 2,values=so2)
P_no2 = TabularCPD('no2', 2,values=no2)
P_co = TabularCPD('co', 2,values=co)
P_pm10 = TabularCPD('pm10', 2,values=pm10)
P_pm25 = TabularCPD('pm25', 2,values=pm25)
P_khai = TabularCPD('khai',2,values=khai)
P_environment_I_o3_so2_no2_co_pm10_pm25_khai = TabularCPD('environment',2,[environment_pro,environment_pro_negative],
                                                            evidence=['o3','so2','no2','co','pm10','pm25','khai'],
                                                            evidence_card=[2,2,2,2,2,2,2]
                                                            )

environment_model.add_cpds(P_o3,P_so2,P_no2,P_co,P_pm10,P_pm25,P_khai, P_environment_I_o3_so2_no2_co_pm10_pm25_khai)                                


### 사회적 요인의 하위 변수 모델링(전기)
P_people = TabularCPD('people',2,values=people)
P_floating = TabularCPD('floating',2,values=floating)
P_EV = TabularCPD('EV',2,values=EV)
P_EVCS = TabularCPD('EVCS',2,values=EVCS)
P_intersection_E = TabularCPD('intersection_E',2,values=intersection_E)
P_distance_E = TabularCPD('distance_E',2,values=distance_E)
P_highway_E = TabularCPD('highway_E',2,values=highway_E)
P_social_E_I_people_floating_EV_EVCS_intersection_E_distance_E_highway_E = TabularCPD('social_E',2,[social_E_pro,social_E_pro_negative],
                                                                                        evidence=['people','floating','EV','EVCS',
                                                                                                  'intersection_E','distance_E','highway_E'],
                                                                                        evidence_card=[2,2,2,2,2,2,2])

social_model_E.add_cpds(P_people,P_floating,P_EV,P_EVCS,P_intersection_E,P_distance_E,P_highway_E,P_social_E_I_people_floating_EV_EVCS_intersection_E_distance_E_highway_E)


### 사회적 요인의 하위 변수 모델링(수소)
P_people = TabularCPD('people',2,values=people)
P_floating = TabularCPD('floating',2,values=floating)
P_LPG = TabularCPD('LPG',2,values=LPG)
P_HV = TabularCPD('HV',2,values=HV)
P_HVCS = TabularCPD('HVCS',2,values=HVCS)
P_intersection_H = TabularCPD('intersection_H',2,values=intersection_H)
P_distance_H = TabularCPD('distance_H',2,values=distance_H)
P_highway_H = TabularCPD('highway_H',2,values=highway_H)
P_social_H_I_people_floating_LPG_HV_HVCS_intersection_H_distance_H_highway_H = TabularCPD('social_H',2,[social_H_pro,social_H_pro_negative],
                                                                                            evidence=['people','floating','LPG','HV',
                                                                                                      'HVCS','intersection_H','distance_H','highway_H'],
                                                                                            evidence_card=[2,2,2,2,2,2,2,2])

social_model_H.add_cpds(P_people,P_floating,P_LPG,P_HV,P_HVCS,P_intersection_H,P_distance_H,P_highway_H,P_social_H_I_people_floating_LPG_HV_HVCS_intersection_H_distance_H_highway_H) 

### 경제적 요인의 하위 변수 모델링(전기)
P_cost_E = TabularCPD('cost_E',2,values=cost_E)
P_area = TabularCPD('area',2,values=area)
P_economic_E_I_cost_E_area = TabularCPD('economic_E',2,[economic_E_pro,economic_E_pro_negative],
                                        evidence=['cost_E','area'],
                                        evidence_card=[2,2])

economic_model_E.add_cpds(P_cost_E,P_area,P_economic_E_I_cost_E_area)   

### 경제적 요인의 하위 변수 모델링(수소)
P_cost_H = TabularCPD('cost_H',2,values=cost_H)
P_LPG_land = TabularCPD('LPG_land',2,values=LPG_land)
P_economic_H_I_cost_H_LPG_land = TabularCPD('economic_H',2,[economic_H_pro,economic_H_pro_negative],
                                            evidence=['cost_H','LPG_land'],
                                            evidence_card=[2,2])

economic_model_H.add_cpds(P_cost_H,P_LPG_land,P_economic_H_I_cost_H_LPG_land)       

### 기술적 요인의 하위 변수 모델링(전기)
P_type = TabularCPD('type',2,values=type)
P_technical_E_I_type = TabularCPD('technical_E',2,[technical_E_pro,technical_E_pro_negative],
                            evidence=['type'],
                            evidence_card=[2])

technical_model_E.add_cpds(P_type,P_technical_E_I_type)


### 기술적 요인의 하위 변수 모델링(수소)
P_complex = TabularCPD('complex',2,values=complex)
P_technical_H_I_complex = TabularCPD('technical_H',2,[technical_H_pro,technical_H_pro_negative],
                                    evidence=['complex'],
                                    evidence_card=[2])

technical_model_H.add_cpds(P_complex,P_technical_H_I_complex)


infer_environment = VariableElimination(environment_model)
infer_social_E = VariableElimination(social_model_E)
infer_social_H = VariableElimination(social_model_H)
infer_economic_E = VariableElimination(economic_model_E)
infer_economic_H = VariableElimination(economic_model_H)
infer_technical_E = VariableElimination(technical_model_E)
infer_technical_H = VariableElimination(technical_model_H)

c:\Users\SAMSUNG\anaconda3\envs\GVCS\lib\site-packages\pgmpy\models\BayesianModel.py:8: FutureWarning:

BayesianModel has been renamed to BayesianNetwork. Please use BayesianNetwork class, BayesianModel will be removed in future.



In [270]:
### 전기차
# 조건부확률 분포표
print(P_environment_I_o3_so2_no2_co_pm10_pm25_khai)                             # 환경적 요인
print(P_social_E_I_people_floating_EV_EVCS_intersection_E_distance_E_highway_E) # 사회적 요인
print(P_economic_E_I_cost_E_area)                                               # 경제적 요인
print(P_technical_E_I_type)                                                     # 기술적 요인

+----------------+---------+-----+---------+---------+---------+
| o3             | o3(0)   | ... | o3(1)   | o3(1)   | o3(1)   |
+----------------+---------+-----+---------+---------+---------+
| so2            | so2(0)  | ... | so2(1)  | so2(1)  | so2(1)  |
+----------------+---------+-----+---------+---------+---------+
| no2            | no2(0)  | ... | no2(1)  | no2(1)  | no2(1)  |
+----------------+---------+-----+---------+---------+---------+
| co             | co(0)   | ... | co(1)   | co(1)   | co(1)   |
+----------------+---------+-----+---------+---------+---------+
| pm10           | pm10(0) | ... | pm10(1) | pm10(1) | pm10(1) |
+----------------+---------+-----+---------+---------+---------+
| pm25           | pm25(0) | ... | pm25(0) | pm25(1) | pm25(1) |
+----------------+---------+-----+---------+---------+---------+
| khai           | khai(0) | ... | khai(1) | khai(0) | khai(1) |
+----------------+---------+-----+---------+---------+---------+
| environment(0) | 1.0   

In [271]:
# 환경적요인의 상태(True:0)를 아는 경우, 각 하위 요인의 확률(True:0,False:1)
print(infer_environment.query(["o3"], evidence={"environment":0},show_progress=False))
print(infer_environment.query(["so2"], evidence={"environment":0},show_progress=False))
print(infer_environment.query(["no2"], evidence={"environment":0},show_progress=False))
print(infer_environment.query(["co"], evidence={"environment":0},show_progress=False))
print(infer_environment.query(["pm10"], evidence={"environment":0},show_progress=False))
print(infer_environment.query(["pm25"], evidence={"environment":0},show_progress=False))
print(infer_environment.query(["khai"], evidence={"environment":0},show_progress=False))

+-------+-----------+
| o3    |   phi(o3) |
+=======+===========+
| o3(0) |    0.8264 |
+-------+-----------+
| o3(1) |    0.1736 |
+-------+-----------+
+--------+------------+
| so2    |   phi(so2) |
+========+============+
| so2(0) |     0.8167 |
+--------+------------+
| so2(1) |     0.1833 |
+--------+------------+
+--------+------------+
| no2    |   phi(no2) |
+========+============+
| no2(0) |     0.9223 |
+--------+------------+
| no2(1) |     0.0777 |
+--------+------------+
+-------+-----------+
| co    |   phi(co) |
+=======+===========+
| co(0) |    0.8508 |
+-------+-----------+
| co(1) |    0.1492 |
+-------+-----------+
+---------+-------------+
| pm10    |   phi(pm10) |
+=========+=============+
| pm10(0) |      0.9023 |
+---------+-------------+
| pm10(1) |      0.0977 |
+---------+-------------+
+---------+-------------+
| pm25    |   phi(pm25) |
+=========+=============+
| pm25(0) |      0.8622 |
+---------+-------------+
| pm25(1) |      0.1378 |
+---------+-------

In [272]:
### 전기차
# 하위요인의 상태를 모르는 경우, 상위 요인의 상태(적합:0, 부적합:1)에 대한 확률
print(infer_environment.query(["environment"],show_progress=False))
print(infer_social_E.query(["social_E"],show_progress=False))
print(infer_economic_E.query(["economic_E"],show_progress=False))
print(infer_technical_E.query(["technical_E"],show_progress=False))

+----------------+--------------------+
| environment    |   phi(environment) |
+================+====================+
| environment(0) |             0.8386 |
+----------------+--------------------+
| environment(1) |             0.1614 |
+----------------+--------------------+
+-------------+-----------------+
| social_E    |   phi(social_E) |
+=============+=================+
| social_E(0) |          0.5731 |
+-------------+-----------------+
| social_E(1) |          0.4269 |
+-------------+-----------------+
+---------------+-------------------+
| economic_E    |   phi(economic_E) |
+===============+===================+
| economic_E(0) |            0.4036 |
+---------------+-------------------+
| economic_E(1) |            0.5964 |
+---------------+-------------------+
+----------------+--------------------+
| technical_E    |   phi(technical_E) |
+================+====================+
| technical_E(0) |             0.3800 |
+----------------+--------------------+
| technical_E(1)

In [273]:
# 하위 요인(o3, no2)의 상태(o3:적합, no2:부적합)가 밝혀진 경우, 환경적 요인의 상태에 대한 확률
result = infer_environment.query(["environment"], evidence={"o3":0, "no2":1}, show_progress=False)
print(result)
result.get_value(environment=0)
# 하위 요인()

+----------------+--------------------+
| environment    |   phi(environment) |
+================+====================+
| environment(0) |             0.7461 |
+----------------+--------------------+
| environment(1) |             0.2539 |
+----------------+--------------------+


0.7461023537134321

## 입지에 대한 확률 업데이트

#### 관측된 데이트에 따른 확률 업데이트(전기차)

In [274]:
def parking_update(name, infer_environment, infer_social_E, infer_economic_E, infer_technical_E):
    parking_area = df_parking[df_parking['주차장명']==name]['주차구획수'].values[0]
    parking_distance = df_parking[df_parking['주차장명']==name]['주변충전소 최단거리'].values[0]
    parking_highway= df_parking[df_parking['주차장명']==name]['주변고속도로까지 최단거리'].values[0]
    parking_intersection = df_parking[df_parking['주차장명']==name]['교차로'].values[0]
    gu = df_parking[df_parking['주차장명']==name]['행정구'].values[0]  
    parking_people = df_busan_people.loc[gu].values[0] 
    parking_floating = df_foottraffic.loc[gu].values[0]
    charger_pro_fast,charger_pro_standard = charge_type(parking_people,parking_floating)
    type = [[charger_pro_fast],[charger_pro_standard]]

    ### 관측된 수치를 기준치로 적합성 판단
    # 0:적합, 1:부적합
    if parking_area > parking_area_standard:
        area_state = 0 
    else:
        area_state = 1
    
    if parking_distance > EV_distance_standard:
        distance_E_state = 0 
    else:
        distance_E_state = 1

    if parking_highway > highway_standard_E:
        highway_E_state = 1 
    else:
        highway_E_state = 0
    
    if parking_intersection > intersection_standard:
        intersection_E_state = 0 
    else:
        intersection_E_state = 1
    
    if parking_people > busan_people_standard:
        people_state = 0 
    else:
        people_state = 1
        
    if parking_floating > foottraffic_standard:
        floating_state = 0 
    else:
        floating_state = 1

    ### 기술적 요인의 하위 변수 업데이트(전기)
    technical_model_E = BayesianModel([('type','technical_E')])

    P_type = TabularCPD('type',2,values=type)
    P_technical_E_I_type = TabularCPD('technical_E',2,[technical_E_pro,technical_E_pro_negative],
                                evidence=['type'],
                                evidence_card=[2])

    technical_model_E.add_cpds(P_type,P_technical_E_I_type)
    infer_technical_E = VariableElimination(technical_model_E)
    
    result_env = infer_environment.query(["environment"],show_progress=False)
    result_soc = infer_social_E.query(["social_E"],evidence={"distance_E":distance_E_state,"highway_E":highway_E_state,"intersection_E":intersection_E_state,
                                                "people":people_state,"floating":floating_state}, show_progress=False)
    result_eco = infer_economic_E.query(["economic_E"],evidence={"area":area_state},show_progress=False)
    result_tec = infer_technical_E.query(["technical_E"],show_progress=False)

    pro_env = result_env.get_value(environment=0)
    pro_soc = result_soc.get_value(social_E=0)
    pro_eco = result_eco.get_value(economic_E=0)
    pro_tec = result_tec.get_value(technical_E=0)

    return pro_env, pro_soc, pro_eco, pro_tec

In [275]:
# parking_list = df_parking['주차장명'].tolist()
# list_env_E = []
# list_soc_E = []
# list_eco_E = []
# list_tec_E = []
# list_total_E = []
# for place in parking_list:
#     pro_env_E, pro_soc_E, pro_eco_E, pro_tec_E = parking_update(place, infer_environment, infer_social_E, infer_economic_E, infer_technical_E)
#     list_env_E.append(round(pro_env_E,6))
#     list_soc_E.append(round(pro_soc_E,6))
#     list_eco_E.append(round(pro_eco_E,6))
#     list_tec_E.append(round(pro_tec_E,6))
#     # 가중치를 통해 최종확률 도출
#     total_pro = 0
#     total_pro += pro_env_E * weight_total[0]
#     total_pro += pro_soc_E * weight_total[1]
#     total_pro += pro_eco_E * weight_total[2]
#     total_pro += pro_tec_E * weight_total[3]
#     list_total_E.append(round(total_pro,6))

In [276]:
# df_parking.insert(14,'기술적요인',list_tec_E)
# df_parking.insert(14,'경제적요인',list_eco_E)
# df_parking.insert(14,'사회적요인',list_soc_E)
# df_parking.insert(14,'환경적요인',list_env_E)
# df_parking.insert(18,'최종확률',list_total_E)
# df_parking.to_csv(parking_file_path,encoding='cp949')

#### 관측된 데이터에 대한 확률 업데이트(수소차)

In [282]:
def LPG_update(name, infer_environment, infer_social_H, infer_economic_H, infer_technical_H):
    gu = df_LPG[df_LPG['업소명']==name]['행정구'].values[0]                              
    LPG_people = df_busan_people.loc[gu].values[0] 
    LPG_floating = df_foottraffic.loc[gu].values[0]
    LPG_land = df_LPG[df_LPG['업소명']==name]['토지비용'].values[0]                    
    LPG_distance = df_LPG[df_LPG['업소명']==name]['주변충전소 최단거리'].values[0]     
    LPG_highway = df_LPG[df_LPG['업소명']==name]['주변고속도로 최단거리'].values[0]    
    LPG_intersection = df_LPG[df_LPG['업소명']==name]['교차로 수'].values[0]     
    LPG_type = charge_type(LPG_people,LPG_floating)

    # 기존 LPG 충전소 위경도 추출
    LPG_lat = df_LPG['위도'].tolist()
    LPG_lon = df_LPG['경도'].tolist()
    location_LPG = list(zip(LPG_lat,LPG_lon))
    location_place = (df_LPG[df_LPG['업소명']==name]['위도'].values[0], df_LPG[df_LPG['업소명']==name]['경도'].values[0]) 
    if location_place in location_LPG:
        complex_pro = 1.0
    else:
        complex_pro = 0.0
    
    ### 관측된 수치를 기준치로 적합성 판단
    # 0:적합, 1:부적합
    if LPG_people > busan_people_standard:
        people_state = 0
    else:
        people_state = 1
    
    if LPG_floating > foottraffic_standard:
        floating_state = 0
    else:
        floating_state = 1
    
    if LPG_land > LPG_land_standard:
        land_state = 1
    else:
        land_state = 0

    if LPG_distance > HV_distance_standard:
        distance_state = 0
    else:
        distance_state = 1
    
    if LPG_highway > highway_standard_H:
        highway_state = 1
    else:
        highway_state = 0
    
    if LPG_intersection > intersection_standard:
        intersection_state = 0
    else:
        intersection_state = 1

    if complex_pro == 1.0:
        complex_state = 0
    else: 
        complex_state = 1
    
    result_env = infer_environment.query(["environment"],show_progress=False)
    result_soc = infer_social_H.query(["social_H"],evidence={"distance_H":distance_state,"highway_H":highway_state,"intersection_H":intersection_state,
                                                             "people":people_state,"floating":floating_state}, show_progress=False)
    result_eco = infer_economic_H.query(["economic_H"],evidence={"LPG_land":land_state},show_progress=False)
    result_tec = infer_technical_H.query(["technical_H"],evidence={"complex":complex_state},show_progress=False)

    pro_env = result_env.get_value(environment=0)
    pro_soc = result_soc.get_value(social_H=0)
    pro_eco = result_eco.get_value(economic_H=0)
    pro_tec = result_tec.get_value(technical_H=0)

    return pro_env, pro_soc, pro_eco, pro_tec

In [287]:
# LPG_list = df_LPG['업소명'].tolist()
# list_env_H = []
# list_soc_H = []
# list_eco_H = []
# list_tec_H = []
# list_total_H = []
# for place in LPG_list:
#     pro_env_H, pro_soc_H, pro_eco_H, pro_tec_H = LPG_update(place, infer_environment, infer_social_H, infer_economic_H, infer_technical_H)
#     list_env_H.append(round(pro_env_H,6))
#     list_soc_H.append(round(pro_soc_H,6))
#     list_eco_H.append(round(pro_eco_H,6))
#     list_tec_H.append(round(pro_tec_H,6))
#     # 가중치를 통해 최종확률 도출
#     total_pro = 0
#     total_pro += pro_env_H * weight_total[0]
#     total_pro += pro_soc_H * weight_total[1]
#     total_pro += pro_eco_H * weight_total[2]
#     total_pro += pro_tec_H * weight_total[3]
#     list_total_H.append(round(total_pro,6))

In [288]:
# df_LPG.insert(13,'기술적요인',list_tec_H)
# df_LPG.insert(13,'경제적요인',list_eco_H)
# df_LPG.insert(13,'사회적요인',list_soc_H)
# df_LPG.insert(13,'환경적요인',list_env_H)
# df_LPG.insert(17,'최종확률',list_total_H)
# df_LPG.to_csv(LPG_file_path,encoding='cp949')

In [291]:
df_parking

,운영형태,행정구,관리기관명,주차장명,주차장관리번호,소재지도로명주소,주차장구분,주차장유형,주차구획수,위도,경도,교차로,주변고속도로까지 최단거리,주변충전소 최단거리,환경적요인,사회적요인,경제적요인,기술적요인,최종확률
0,구(군)직영,사상구,부산광역시 사상구청,감전동주변,2010000013,사상구 감전동 589-2,공영,노외,26,35.154108,128.980071,239,4346.343726,0.061228,0.838573,0.643981,0.252453,0.488554,0.532397
1,민간위탁,사상구,부산광역시 사상구청,엄궁초등학교지하주차장,2010000016,사상구 엄궁로191번길 23,공영,노외,93,35.128446,128.973403,85,7240.940505,0.112529,0.838573,0.493981,0.252453,0.488554,0.472397
2,구(군)직영,사상구,부산광역시 사상구청,일신주택주변,2011000003,사상구 주례3동 545-3,공영,노외,10,35.145207,129.000672,165,5584.115172,0.247823,0.838573,0.643981,0.252453,0.488554,0.532397
3,구(군)직영,사상구,부산광역시 사상구청,주례중학교주변,2012000004,사상구 주례2동 82-13,공영,노외,8,35.154384,129.011131,276,5040.598424,0.094006,0.838573,0.643981,0.252453,0.488554,0.532397
4,민간위탁,사상구,부산광역시 사상구청,모라1동행정복지센터주변,2007001725,사상구 백양대로 949-6,공영,노상,156,35.171044,128.988085,249,2512.034932,0.190308,0.838573,0.643981,0.752453,0.488554,0.682397
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
468,민간위탁,중구,부산광역시 중구청,중앙동 노외공영주차장,2009000002,중구 대청로135번길 36,공영,노외,29,35.104229,129.034810,593,10976.446670,0.143217,0.838573,0.443981,0.252453,0.393172,0.438090
469,민간운영,기장군,부산광역시 기장군청,1구역 2소,2008000002,기장군 기장읍 280-2,공영,노상,12,35.222141,129.206947,19,162.104257,1.603011,0.838573,0.493981,0.252453,0.327783,0.448282
470,민간운영,기장군,부산광역시 기장군청,1구역 3소,2008000003,기장군 기장읍 444-1,공영,노상,13,35.222141,129.206947,19,162.104257,1.603011,0.838573,0.493981,0.252453,0.327783,0.448282
471,민간운영,기장군,부산광역시 기장군청,3구역 1소,2008000006,기장군 기장읍 418-12,공영,노상,24,35.222141,129.206947,19,162.104257,1.603011,0.838573,0.493981,0.252453,0.327783,0.448282


In [292]:
df_LPG

,행정시,행정구,업소명,소재지,관리구분,위도,경도,면적,공시지가,토지비용,주변충전소 최단거리,주변고속도로 최단거리,교차로 수,환경적요인,사회적요인,경제적요인,기술적요인,최종확률
0,부산,강서구,개인택시신공항충전소,부산 강서구 경전철로188번길 95,자동차충전,35.198669,128.966594,2545.0,2144000.0,5.456480e+09,6.175720,0.815596,52,0.838573,0.501317,0.768969,1,0.707003
1,부산,강서구,부산공항LPG충전소,부산 강서구 공항로 459,자동차충전,35.149064,128.951891,3105.0,1174000.0,3.645270e+09,3.270797,5.631472,16,0.838573,0.301317,0.768969,1,0.627003
2,부산,강서구,신항만에너지,부산 강서구 낙동남로 507,자동차충전,35.112947,128.875260,3105.0,1174000.0,3.645270e+09,2.907968,11.968669,76,0.838573,0.301317,0.768969,1,0.627003
3,부산,강서구,(주)지원네트웍스 부산강서LPG충전소,부산 강서구 낙동북로 310,자동차충전,35.212507,128.962572,1396.0,1140000.0,1.591440e+09,7.752939,0.864172,32,0.838573,0.501317,0.768969,1,0.707003
4,부산,강서구,신명지에너지,부산 강서구 르노삼성대로 560,자동차충전,35.097554,128.927351,481.0,700300.0,3.368443e+08,7.676891,11.748654,159,0.838573,0.551317,0.768969,1,0.727003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,부산,연제구,부산개인택시엘피지충전소,부산 연제구 월드컵대로 399,자동차충전,35.186013,129.055974,9212.0,688600.0,6.343383e+09,7.648635,5.612178,146,0.838573,0.651317,0.268969,1,0.617003
59,부산,영도구,(주)태종에너지,부산 영도구 영선대로 41,자동차충전,35.084996,129.040033,1610.2,2679000.0,4.313726e+09,8.277014,13.118878,255,0.838573,0.551317,0.768969,1,0.727003
60,부산,영도구,부산에너지,부산 영도구 해양로 160,용기+자동차+탱크+13kg용기,35.091275,129.069484,5801.0,989400.0,5.739509e+09,9.604391,13.821871,67,0.838573,0.401317,0.768969,1,0.667003
61,부산,영도구,개인택시 영도LPG충전소,부산 영도구 해양로 296,자동차충전,35.082176,129.076429,1818.0,349800.0,6.359364e+08,10.740567,14.353595,98,0.838573,0.401317,0.768969,1,0.667003
